#  STEP 1. SETUP

In [1]:
# if the script finishes very quickly (and generates an empty excel file), click run again
# if the script errors on the "Login Cell" (added a comment to indicate which cell that is below), set IS_HEADLESS to "False" and run again. The scraper will automatically launch a page and attempt to login to LinkedIn. It's likely erroring because LinkedIn is asking for a captcha to verify the user is not a bot. Solve the captch/challenge and login. Once successfully logged in, set IS_HEADLESS back to "True" and run again.

import pickle
import csv
import os
import random
import time
from linkedin_scraper import actions
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException, NoSuchElementException, ElementClickInterceptedException

# for google docs upload
import pickle
import gspread
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build
from datetime import datetime

IS_HEADLESS = False

LINKEDIN_EMAIL = ""
LINKEDIN_PASSWORD = ""


AIRCRAFT_CARRIERS = [
    "Square",
    "Palantir",
    "Verkada",
    "Retool",
    "Doordash",
    "Scale AI",
    "Brex",
    "Ramp",
    "Rippling",
    "Zapier",
    "Dropbox",
    "Anduril",
    "SpaceX",
    "Wiz",
    "Cruise",
    "Datadog",
    "Databricks",
    "Figma",
    "Notion",
    "Abnormal Security",
    "Athelas",
    "Commure",
]

FLAGGED_ROLES = [
    "forward deployed engineer",
    "vertical lead",
    "tech lead",
    "staff software engineer",
    "staff swe",
    "principal software engineer",
    "principal swe",
    "engineering manager",
    "staff product manager",
    "staff pm",
    "principal product manager",
    "principal pm",
    "chief of staff",
    "head of engineering",
    "director of engineering",
    "head of product",
    "director of product",
    "head",
    "president",
    "vice president",
    "vp",
    "director"
]

## Login Helper Function

In [2]:
# IF THIS CELL ERRORS DUE TO CAPTCHA, do this:
# manually complete the captcha, click on the next cell, and select Run menu, select "run selected cell and all below" 

def instantiate_driver():
    options = FirefoxOptions()
    if IS_HEADLESS:
        options.add_argument("--headless")
    driver = webdriver.Firefox(options=options)
    
    try:
        actions.login(driver, LINKEDIN_EMAIL, LINKEDIN_PASSWORD)
    except Exception as e:
        print("Error logging in. Please complete the captcha challenge and login manually.")
        print(e)

    time.sleep(15)
    return driver

## General Helper Functions

In [3]:

"""
Wait for an element to be present on the page and return it.

Parameters:
- driver: The WebDriver instance
- by: The method to locate the element (default: By.CLASS_NAME)
- name: The name or identifier of the element to wait for
- base: The base element to search from (default: None, which uses the driver)
- timeout: Maximum time to wait for the element (default: 180 seconds)

Returns:
- The WebElement if found
- None if the element is not found within the timeout period
"""
def wait_for_element_to_load(driver, by=By.CLASS_NAME, name="pv-top-card", base=None, timeout=20):
    base = base or driver
    try:
        element = WebDriverWait(base, timeout).until(
            EC.presence_of_element_located((by, name))
        )
        return element
    except TimeoutException:
        print(f"Timed out waiting for element: {by}={name}")
        return None
    except Exception as e:
        print(f"An error occurred while waiting for element {by}={name}: {str(e)}")
        return None


def element_exists(item, by, value):
    try:
        item.find_element(by, value)
        return True
    except:
        return False
    

def readPickle(input):
    results = []
    with open(input, 'rb') as f:
        try:
            results = pickle.load(f)
            print("Loaded pickle file: " + input + " with " + str(len(results)) + " results")
        except Exception as e:
            print(e)
            print("Error loading the pickle file: " + input)
    return results

def writePickle(output, data):
    with open(output, 'wb') as f:
        try:
            pickle.dump(data, f)
            print("Saved pickle file: " + output + " with " + str(len(data)) + " results")
        except Exception as e:
            print(e)
            print("Error saving the pickle file: " + output)


## Sales Navigator Helper Functions

In [4]:
def check_and_load_pickles(historyPickle, toScrapePickle):
    scraped_urls = []
    to_scrape_urls = []
    scraped_urls = readPickle(historyPickle)
    to_scrape_urls = readPickle(toScrapePickle)
    return scraped_urls, to_scrape_urls
    

def deduplicate(toScrapePickle, to_scrape_urls, already_scraped_urls):
    originalCount = len(to_scrape_urls)
    to_scrape_urls = list(set(to_scrape_urls))
    to_scrape_urls = [url for url in to_scrape_urls if url not in already_scraped_urls]
    print(to_scrape_urls, already_scraped_urls)
    print(f"Removed {originalCount - len(to_scrape_urls)} duplicates")
    writePickle(toScrapePickle, to_scrape_urls)
    

# Scrolls the page to bring the specified profile element into view
# Returns True if successful, False if an error occurs
def scroll_to_profile(driver, profile):
    try:
        driver.execute_script("arguments[0].scrollIntoView();", profile)
        return True
    except Exception as e:
        print(f"Error scrolling to profile: {e}")
        return False

# Clicks on the profile element to open its details
# Returns True if successful, False if an error occurs
def click_profile(profile):
    try:
        salesNavOpenProfileButton = profile.find_element(By.CLASS_NAME, "artdeco-entity-lockup__title")
        salesNavOpenProfileButton.click()
        return True
    except (NoSuchElementException, ElementClickInterceptedException) as e:
        print(f"Error clicking profile: {e}")
        return False

# Clicks the three dots button to open the dropdown menu
# Returns True if successful, False if an error occurs
def click_three_dots_button(driver):
    try:
        actionContainer = driver.find_element(By.CLASS_NAME, "_actions-container_1dg5u8")
        threeDotsButton = actionContainer.find_element(By.CLASS_NAME, "_icon_ps32ck")
        threeDotsButton.click()
        return True
    except (NoSuchElementException, ElementClickInterceptedException) as e:
        print(f"Error clicking three dots button")
        return False

# Retrieves the LinkedIn URL from the dropdown menu
# Returns the URL if successful, None if an error occurs
def get_linkedin_url(driver):
    try:
        dropdownContainer = driver.find_element(By.CLASS_NAME, "_visible_x5gf48")
        normalLinkedInUrl = dropdownContainer.find_elements(By.TAG_NAME, "a")[1].get_attribute("href")
        return normalLinkedInUrl
    except (NoSuchElementException, IndexError) as e:
        print(f"Error getting LinkedIn URL: {e}")
        return None

# Closes the profile popout
# Returns True if successful, False if an error occurs
def close_popout(driver):
    try:
        header = driver.find_element(By.CLASS_NAME, "_inline-sidesheet-header-actions_1cn7lg")
        button = header.find_elements(By.CLASS_NAME, "_button_ps32ck")[1]
        button.click()
        return True
    except (NoSuchElementException, ElementClickInterceptedException, IndexError) as e:
        print(f"Error closing popout: {e}")
        return False

# Navigates to the next page of search results
# Returns True if successful, False if there are no more pages or an error occurs
def navigate_to_next_page(driver):
    try:
        nextPageButton = driver.find_element(By.CLASS_NAME, "artdeco-pagination__button--next")
        nextPageButton.click()
        return True
    except (NoSuchElementException, ElementClickInterceptedException) as e:
        print(f"No more pages or error navigating: {e}")
        return False

# Main scraping function
def scrape_profiles(driver, SALES_NAV_SEARCH_URL, already_scraped_urls, to_scrape_urls, toScrapePickle):
    try:
        # reinstantiate_driver(driver)
        driver.get(SALES_NAV_SEARCH_URL)
        
        while True:
            wait_for_element_to_load(driver, By.ID, "search-results-container")
            profiles = driver.find_elements(By.CLASS_NAME, "artdeco-list__item")
            
            for profile in profiles:
                if not scroll_to_profile(driver, profile):
                    continue

                if not wait_for_element_to_load(driver, By.CLASS_NAME, "artdeco-entity-lockup__title"):
                    continue

                if not click_profile(profile):
                    continue
                time.sleep(2 + random.random() * 6)


                if not wait_for_element_to_load(driver, By.CLASS_NAME, "_actions-container_1dg5u8"):
                    continue

                if not click_three_dots_button(driver):
                    continue
                time.sleep(2 + random.random() * 6)


                if not wait_for_element_to_load(driver, By.CLASS_NAME, "_visible_x5gf48"):
                    continue

                normalLinkedInUrl = get_linkedin_url(driver)
                if normalLinkedInUrl:
                    if normalLinkedInUrl in already_scraped_urls:
                        print("Skipping (already scraped): " + normalLinkedInUrl)
                    else:
                        to_scrape_urls.append(normalLinkedInUrl)
                        writePickle(toScrapePickle, to_scrape_urls)
                        print("Successfully scraped: " + normalLinkedInUrl)

                if not close_popout(driver):
                    continue

            next_button = wait_for_element_to_load(driver, By.CLASS_NAME, "artdeco-pagination__button--next")
            if not next_button or not next_button.is_enabled():
                break

            next_button.click()
            time.sleep(2 + random.random() * 6)


            if not wait_for_element_to_load(driver, By.CLASS_NAME, "artdeco-list__item"):
                break
    except Exception as e:
        raise e
    return to_scrape_urls

## Profile Scraping Helper Functions

In [5]:
def get_experiences(driver):
    scraped_experiences = []

    try:
        wait_for_element_to_load(driver, By.CSS_SELECTOR, "li.pvs-list__paged-list-item.artdeco-list__item.pvs-list__item--line-separated.pvs-list__item--one-column")
        experience_items = driver.find_elements(By.CSS_SELECTOR, "li.pvs-list__paged-list-item.artdeco-list__item.pvs-list__item--line-separated.pvs-list__item--one-column")  
        # it was still wokring but these conditions weren't present
        # experience_items = driver.find_elements(By.CSS_SELECTOR, "li.artdeco-list__item")
        if len(experience_items) > 0:
            for item in experience_items:
                title = None
                company = None
                dates = None
                location = None
                summary = None

                #multi_experience_selector = ".pvs-entity__sub-components"

                # For multiple experiences at one company
                if element_exists(item, By.CSS_SELECTOR, ".pvs-entity__sub-components") and \
                   element_exists(item, By.CSS_SELECTOR, ".fBLvVtHeoxHyebPSvKtIPPVdFQRcOqKdA"): # (CSS selector string for the bullet point)
                    company_selector = "div.display-flex.flex-column.full-width.align-self-center div.display-flex.flex-wrap.align-items-center.full-height span.visually-hidden"
                    if element_exists(item, By.CSS_SELECTOR, company_selector):
                        company = item.find_element(By.CSS_SELECTOR, company_selector).text

                    roles = item.find_elements(By.CSS_SELECTOR, "li.pvs-list__paged-list-item.pvs-list__item--one-column")
                    for role in roles:
                        title_selector = "div.display-flex.flex-wrap.align-items-center.full-height span.visually-hidden"
                        if element_exists(role, By.CSS_SELECTOR, title_selector):
                            title = role.find_element(By.CSS_SELECTOR, title_selector).text
                        
                        dates_and_location_selector = "span.t-14.t-normal.t-black--light"
                        if element_exists(role, By.CSS_SELECTOR, dates_and_location_selector):
                            dates_and_location_elements = role.find_elements(By.CSS_SELECTOR, dates_and_location_selector)
                            dates = dates_and_location_elements[0].find_element(By.CSS_SELECTOR, "span.visually-hidden").text
                            if (len(dates_and_location_elements) > 1):
                                location = dates_and_location_elements[1].find_element(By.CSS_SELECTOR, "span.visually-hidden").text

                        summary_selector = "div.display-flex.align-items-center.t-14.t-normal.t-black span.visually-hidden"
                        if element_exists(role, By.CSS_SELECTOR, summary_selector):
                            summary = role.find_element(By.CSS_SELECTOR, summary_selector).text

                        experience = {}
                        if title:
                            experience["title"] = title
                        if company:
                            experience["company"] = company
                        if dates:
                            experience["dates"] = dates
                        if location:
                            experience["location"] = location
                        if summary:
                            experience["summary"] = summary
                        print(experience)
                        scraped_experiences.append(experience)


                # For single experience at one company
                else:
                    title_selector = "div.display-flex.flex-wrap.align-items-center.full-height span.visually-hidden"
                    if element_exists(item, By.CSS_SELECTOR, title_selector):
                        title = item.find_element(By.CSS_SELECTOR, title_selector).text
                    
                    company_selector = "span.t-14.t-normal span.visually-hidden"
                    if element_exists(item, By.CSS_SELECTOR, company_selector):
                        company = item.find_element(By.CSS_SELECTOR, company_selector).text
                    
                    dates_and_location_selector = "span.t-14.t-normal.t-black--light"
                    if element_exists(item, By.CSS_SELECTOR, dates_and_location_selector):
                        dates_and_location_elements = item.find_elements(By.CSS_SELECTOR, dates_and_location_selector)
                        dates = dates_and_location_elements[0].find_element(By.CSS_SELECTOR, "span.visually-hidden").text
                        if (len(dates_and_location_elements) > 1):
                            location = dates_and_location_elements[1].find_element(By.CSS_SELECTOR, "span.visually-hidden").text
                    
                    summary_selector = "div.display-flex.align-items-center.t-14.t-normal.t-black span.visually-hidden"
                    if element_exists(item, By.CSS_SELECTOR, summary_selector):
                        summary = item.find_element(By.CSS_SELECTOR, summary_selector).text
                    
                    experience = {}
                    print(title, company, dates, location, summary)
                    if title:
                        experience["title"] = title
                    if company:
                        experience["company"] = company
                    if dates:
                        experience["dates"] = dates
                    if location:
                        experience["location"] = location
                    if summary:
                        experience["summary"] = summary
                    print(experience)
                    scraped_experiences.append(experience)
            print("Successfully scraped experiences")
        else:
            print("No experiences found")

    except Exception as e:
        print(e)
        print("ERROR: No experiences found")

    return scraped_experiences


# Helper function to scrape education
# Returns scraped education if successful, otherwise returns empty list
def get_education(driver):
    scraped_education = []

    try:
        wait_for_element_to_load(driver, By.CSS_SELECTOR, "li.pvs-list__paged-list-item.artdeco-list__item.pvs-list__item--line-separated.pvs-list__item--one-column")
        education_items = driver.find_elements(By.CSS_SELECTOR, "li.pvs-list__paged-list-item.artdeco-list__item.pvs-list__item--line-separated.pvs-list__item--one-column")

        if len(education_items) > 0:
            for item in education_items:
                hidden_spans = item.find_elements(By.CSS_SELECTOR, "span.visually-hidden")
                education_texts = [span.text for span in hidden_spans]
                education = {
                    "school": education_texts[0],
                }
                
                if len(education_texts) > 1:
                    education["degree"] = education_texts[1]
                if len(education_texts) > 2:
                    education["dates"] = education_texts[2]
                if len(education_texts) > 3:
                    education["remaining"] = (", ").join(education_texts[5:])
                scraped_education.append(education)

            print("Successfully scraped education")
        else:
            print("No education found")

    except Exception as e:
        print(e)
        print("ERROR: No education found")

    return scraped_education


# Helper function to scrape degree of connection and mutuals
# Returns scraped degree of connection and mutuals if successful, otherwise returns N/A
def get_degree_of_connection_and_mutuals(driver):
    scraped_profile_dist = "4+"
    scraped_mutuals = "N/A"

    try:
        scraped_profile_dist = wait_for_element_to_load(driver, By.CSS_SELECTOR, "span.dist-value").text
        if scraped_profile_dist == "1st" or scraped_profile_dist == "2nd":
            try:
                span_element = wait_for_element_to_load(driver, By.CSS_SELECTOR, "span.t-normal.t-black--light.t-14.hoverable-link-text")
                scraped_mutuals = span_element.text.split('\n')[0]
                print("Successfully found mutual connections: " + scraped_mutuals)
            except:
                print("ERROR: mutuals not found")
        print("Successfully scraped degree of connection: " + scraped_profile_dist)

    except Exception as e:
        print(e)
        print("No degree of connection found")

    return scraped_profile_dist, scraped_mutuals

# Helper function to scrape description
# Returns scraped description if successful, otherwise returns N/A
def get_description(driver):
    scraped_description = "N/A"
    
    try:
        scraped_description = driver.find_element(By.CLASS_NAME, "text-body-medium.break-words").text
        print("Successfully scraped description: " + scraped_description)
    except Exception as e:
        print(e)
        print("ERROR: description not found")
        
    return scraped_description

# Main function to scrape degree of connection and mutuals
# Returns scraped degree of connection and mutuals if successful, otherwise returns N/A
def scrape_profile(driver, scraped_link):

    # Scrape Name
    driver.get(scraped_link)
    scraped_profile_name = wait_for_element_to_load(driver, By.CSS_SELECTOR, "h1[class*='inline t-24 v-align-middle break-words']").text
    time.sleep(2 + random.random() * 6)

    # Scrape Experiences
    experiences_url = os.path.join(scraped_link, "details/experience")
    driver.get(experiences_url)
    scraped_experiences = get_experiences(driver)
    time.sleep(2 + random.random() * 6)


    # Scrape Education
    education_url = os.path.join(scraped_link, "details/education")
    driver.get(education_url)
    scraped_education = get_education(driver)
    time.sleep(2 + random.random() * 6)


    # Scrape degree of connection and mutuals if available
    driver.get(scraped_link)
    scraped_profile_dist, scraped_mutuals = get_degree_of_connection_and_mutuals(driver)
    time.sleep(2 + random.random() * 6)

    # Scrape description
    scraped_description = get_description(driver)
    time.sleep(2 + random.random() * 6)


    # Scape profile link
    scraped_link = driver.current_url
    print("Successfully scraped profile link: " + scraped_link)
    time.sleep(2 + random.random() * 6)

    profile = {}
    profile["name"] = scraped_profile_name
    profile["experiences"] = scraped_experiences
    profile["education"] = scraped_education
    profile["profile_dist"] = scraped_profile_dist
    profile["mutuals"] = scraped_mutuals
    profile["profile_description"] = scraped_description
    profile["profile_link"] = scraped_link

    print("\nSuccess!")
    print(f"Name: {profile['name']}")
    print("Experiences:")
    for exp in profile["experiences"]:
        print(f"  Title: {exp.get('title', '')}")
        print(f"  Company: {exp.get('company', '')}")
        print(f"  Dates: {exp.get('dates', '')}")
        print(f"  Location: {exp.get('location', '')}")
        print(f"  Summary: {exp.get('summary', '')}")
    print("Education:")
    for edu in profile["education"]:
        print(f"  School: {edu.get('school', '')}")
        print(f"  Degree: {edu.get('degree', '')}")
        print(f"  Dates: {edu.get('dates', '')}")
        print(f"  Remaining: {edu.get('remaining', '')}")
    print(f"Profile Distance: {profile['profile_dist']}")
    print(f"Mutuals: {profile['mutuals']}")
    print(f"Description: {profile['profile_description']}")
    print(f"Link: {profile['profile_link']}\n")

    return profile


def scrape_all_profiles(driver, historyPickle, toScrapePickle, resultsPickle, failedURLsTextFile):
    results = readPickle(resultsPickle)
    scraped_urls = []
    history = []
    to_scrape_urls = []
    to_scrape_urls = readPickle(toScrapePickle)
        
    totalCount = len(to_scrape_urls)

    print(f'# of profiles to scrape: {totalCount}')
    

    for i in range(len(to_scrape_urls) - 1, -1, -1):
        url = to_scrape_urls[i]
        print(f'At index: {totalCount - i} - url: {url}')
        history = readPickle(historyPickle)

        try:
            profile = scrape_profile(driver, url)
            if profile != None:
                print("saving profile info")
                results.append(profile)
                writePickle(resultsPickle, results)
                
                print("adding to history")
                history.append(url)
                writePickle(historyPickle, history)
                
                print("recording scraped url")
                scraped_urls.append(url)
            else:
                print("profile filtered out")
            
            print("removing from to-scrape")
            to_scrape_urls.remove(url)
            writePickle(toScrapePickle, to_scrape_urls)

        except Exception as e:
            print(e)
            print('Failed to scrape profile: ', url)

            with open(failedURLsTextFile, 'a') as f:
                f.write(url + '\n')

        print(((totalCount - i)/totalCount) * 100, '% Done - at index:', totalCount - i)
        print('\n----------------------------------------------------------------------------------------------------------\n')
    print("Successfully scraped: " + str(len(results)) + " profiles")
    return results

## Exporting Helper Functions

In [6]:
def authenticate():
    scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
    creds = Credentials.from_service_account_file('google_auth.json', scopes=scope)
    client = gspread.authorize(creds)
    drive_service = build('drive', 'v3', credentials=creds)
    return client, drive_service

def create_sheet(folder_id):
    client, drive_service = authenticate()

    # Create sheet
    today_date = datetime.today().strftime('%Y-%m-%d')
    sheet_name = f'Linkedin Scraper Results {today_date}'
    spreadsheet = client.create(sheet_name)

    # Add additional worksheets
    spreadsheet.add_worksheet(title='Aircraft Profiles', rows=1000, cols=100)
    spreadsheet.add_worksheet(title='Main', rows=1000, cols=100)
    spreadsheet.add_worksheet(title='Infra', rows=1000, cols=100)
    spreadsheet.add_worksheet(title='Portfolio', rows=1000, cols=100)
    spreadsheet.add_worksheet(title='Unicorn', rows=1000, cols=100)
    spreadsheet.add_worksheet(title='Acquired', rows=1000, cols=100)

    # Delete the default sheet
    default_sheet = spreadsheet.sheet1
    spreadsheet.del_worksheet(default_sheet)

    # Move the spreadsheet to the specified folder
    file_id = spreadsheet.id
    drive_service.files().update(fileId=file_id, addParents=folder_id, removeParents='root').execute()

    return spreadsheet


def export_sheet(profiles, spreadsheet, worksheet_title):
    # Access the specified worksheet by title
    sheet = spreadsheet.worksheet(worksheet_title)
    
    # Prepare the header for the combined sheet
    max_edu = 3
    max_exp = max(len(profile['experiences']) for profile in profiles)
    header = ["Link", "Name", "Description", "Distance", "Mutuals"]
    header.extend([f"Edu{i}" for i in range(max_edu)])
    for i in range(max_exp):
        header.extend([f"Exp{i}", f"Exp{i} Summary"])
    sheet.append_row(header)

    # Prepare the rows
    for profile in profiles:
        row = [profile['profile_link'], profile['name'], profile['profile_description'], profile['profile_dist'], profile['mutuals']]
        
        # Add education data
        edu_entries = []
        for edu in profile.get('education', [])[:max_edu]:
            edu_entry = ""
            if 'school' in edu:
                edu_entry += f"School: {edu['school']}\n"
            if 'degree' in edu:
                edu_entry += f"Degree: {edu['degree']}\n"
            if 'dates' in edu:
                edu_entry += f"Dates: {edu['dates']}\n"
            edu_entries.append(edu_entry.strip())
        edu_entries.extend([''] * (max_edu - len(edu_entries)))
        row.extend(edu_entries)
        
        # Add experiences data
        for exp in profile.get('experiences', []):
            exp_entry = ""
            if 'title' in exp:
                exp_entry += f"Title: {exp['title']}\n"
            if 'company' in exp:
                exp_entry += f"Company: {exp['company']}\n"
            if 'dates' in exp:
                exp_entry += f"Dates: {exp['dates']}\n"
            if 'location' in exp:
                exp_entry += f"Location: {exp['location']}\n"
            row.append(exp_entry.strip())
            row.append(exp.get('summary', '').strip())
        # Pad with empty strings if there are fewer experience entries than the maximum
        row.extend(['', ''] * (max_exp - len(profile.get('experiences', []))))

        # Append the row to the sheet
        sheet.append_row(row)
        time.sleep(1)

    # Construct and return the specific worksheet's URL
    worksheet_url = f"{spreadsheet.url}#gid={sheet.id}"
    return worksheet_url


# STEP 2. CREATE NEW GOOGLE SHEET
### Run only ONCE for each scraper run

In [7]:
# Create google sheet
folder_id = '1vtHm_M0hwADZX76MLHXNL5WqhmE310Vo'
spreadsheet = create_sheet(folder_id)

# New Query: Aircraft Carrier Profiles

### Setup

In [8]:
historyPickle = 'db/history.pickle'
toScrapePickle = 'db/main_to_scrape.pickle'
resultsPickle = 'db/main_results.pickle'
failedURLsTextFile = 'db/main_failed_urls.txt'

SALES_NAV_SEARCH_URL = """https://www.linkedin.com/sales/search/people?savedSearchId=1890984140&sessionId=fnpRgMn2S96i3KXlFoDYTg%3D%3D"""

### Part 1: Sales Nav Scraper

In [ ]:
# Sales Nav Scraping
driver = instantiate_driver()
already_scraped_urls, to_scrape_urls = check_and_load_pickles(historyPickle, toScrapePickle)
to_scrape_urls = scrape_profiles(driver, SALES_NAV_SEARCH_URL, already_scraped_urls, to_scrape_urls, toScrapePickle)
print(str(len(to_scrape_urls)) + "profiles identified for scraping") 

### Part 2: Profile scraper, export data to Google Sheets, and reset results pickle

In [ ]:
# Profile Scraping
deduplicate(toScrapePickle, to_scrape_urls, already_scraped_urls)
results = scrape_all_profiles(driver, historyPickle, toScrapePickle, resultsPickle, failedURLsTextFile)


# Export results to Google Sheets
worksheet_title = 'Aircraft Profiles'
resultsPickle = 'db/main_results.pickle'
results = readPickle(resultsPickle)
if results:
    worksheet_url = export_sheet(results, spreadsheet, worksheet_title)
    print(f"Worksheet updated: {worksheet_url}")
else:
    print("No data to export.")


# Reset results
results = []
writePickle(resultsPickle, results)
driver.close()

# QUERY 1: MAIN COMPANIES

### Setup

In [8]:
historyPickle = 'db/history.pickle'
toScrapePickle = 'db/main_to_scrape.pickle'
resultsPickle = 'db/main_results.pickle'
failedURLsTextFile = 'db/main_failed_urls.txt'

SALES_NAV_SEARCH_URL = """
https://www.linkedin.com/sales/search/people#query=(spellCorrectionEnabled%3Atrue%2CrecentSearchParam%3A(id%3A3651379850%2CdoLogHistory%3Atrue)%2Cfilters%3AList((type%3ACURRENT_COMPANY%2Cvalues%3AList((id%3Aurn%253Ali%253Aorganization%253A18583501%2Ctext%3AStealth%2520Startup%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A18016269%2Ctext%3AStealth%2520Mode%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A24732%2Ctext%3AStealth%2520Mode%2520Startup%2520Company%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A91313799%2Ctext%3AStealth%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A31285650%2Ctext%3ANewCo%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A584065%2Ctext%3ANewco%2520Stealth%2520Mode%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A1832989%2Ctext%3ANewco%2520-%2520still%2520in%2520stealth%2520mode%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))))%2C(type%3APAST_COMPANY%2Cvalues%3AList((id%3Aurn%253Ali%253Aorganization%253A1441%2Ctext%3AGoogle%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A17988315%2Ctext%3ARippling%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A20708%2Ctext%3APalantir%2520Technologies%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A3809194%2Ctext%3AFront%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A10037698%2Ctext%3AMeesho%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A100377923%2Ctext%3AWarpStream%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A10043614%2Ctext%3ASnyk%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A100996178%2Ctext%3ARitual%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A101156365%2Ctext%3AManta%2520Network%2520%2528MANTA%2529%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A102840241%2Ctext%3ANagomi%2520Security%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A10454372%2Ctext%3AGong%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A10489311%2Ctext%3AVAST%2520Data%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A10602239%2Ctext%3APaystack%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A10607336%2Ctext%3AChainalysis%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A10614939%2Ctext%3APrisma%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A10638445%2Ctext%3APeople.ai%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A10639445%2Ctext%3ACyberhaven%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A10643112%2Ctext%3AThe%2520Athletic%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A10653469%2Ctext%3ALattice%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A10667%2Ctext%3AMeta%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A10676004%2Ctext%3AFinix%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A10687033%2Ctext%3APrivacera%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A10813156%2Ctext%3AGroww%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A10893210%2Ctext%3Adbt%2520Labs%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A11062162%2Ctext%3AGrafana%2520Labs%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A11130470%2Ctext%3AOpenAI%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A11193683%2Ctext%3AHugging%2520Face%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A11199396%2Ctext%3ARo%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A11247457%2Ctext%3ASolugen%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A1152516%2Ctext%3AFutureAdvisor%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A11537059%2Ctext%3AAlgorand%2520Technologies%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A11741116%2Ctext%3ARunway%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A11869260%2Ctext%3ARetool%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A1189697%2Ctext%3AOptimizely%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A1279612%2Ctext%3APresto%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A12996946%2Ctext%3AChartHop%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A13018048%2Ctext%3AClay%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A1301808%2Ctext%3ADocker%252C%2520Inc%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A13449964%2Ctext%3APolygon%2520Labs%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A14059168%2Ctext%3AMaterialize%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A1406226%2Ctext%3ARamp%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A14824547%2Ctext%3AFireblocks%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A1523656%2Ctext%3AUiPath%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A1594050%2Ctext%3AGoogle%2520DeepMind%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A16181286%2Ctext%3AVercel%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A16181958%2Ctext%3AProtocol%2520Labs%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A167842%2Ctext%3AWeebly%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A17900793%2Ctext%3AWaymo%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A17932068%2Ctext%3ALacework%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A17963177%2Ctext%3AMezmo%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A17996602%2Ctext%3ABear%2520Flag%2520Robotics%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A17997940%2Ctext%3AHoneylove%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A17998520%2Ctext%3AScale%2520AI%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A18013280%2Ctext%3AFaire%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A1815218%2Ctext%3AUber%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A18226665%2Ctext%3AAxonius%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A18264732%2Ctext%3AdYdX%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A18289391%2Ctext%3AChromatic%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A18293159%2Ctext%3AAnduril%2520Industries%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A18309569%2Ctext%3ASemgrep%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A18374423%2Ctext%3AMomentus%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A18436813%2Ctext%3ASubstack%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A18453134%2Ctext%3ASolana%2520Labs%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A18468937%2Ctext%3AFortressIQ%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A18505670%2Ctext%3ABrex%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A18542592%2Ctext%3AReplit%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A18586257%2Ctext%3AAbnormal%2520Security%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A18593641%2Ctext%3AWeights%2520%2526%2520Biases%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A18648301%2Ctext%3A1Password%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A18695169%2Ctext%3AGatsby%2520%2528A%2520Netlify%2520Company%2529%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A18742807%2Ctext%3ATRM%2520Labs%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A18769344%2Ctext%3AModern%2520Treasury%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A18777798%2Ctext%3ACribl%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A18805347%2Ctext%3ASublime%2520Security%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A18842066%2Ctext%3ABraintrust%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A18850692%2Ctext%3APersona%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A18922914%2Ctext%3ADeel%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A18965190%2Ctext%3APanther%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A19029173%2Ctext%3AMiro%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A19055959%2Ctext%3ACensus%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A19107985%2Ctext%3AMercury%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A2014725%2Ctext%3ARedis%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A20299330%2Ctext%3APinecone%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A20313069%2Ctext%3AMonte%2520Carlo%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A2135371%2Ctext%3AStripe%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A2169323%2Ctext%3AFivestars%2520by%2520SumUp%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A2241894%2Ctext%3AGinkgo%2520Bioworks%252C%2520Inc.%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A2320329%2Ctext%3ATwitch%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A2326545%2Ctext%3ASmartAsset%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A24024765%2Ctext%3ACohere%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A2418251%2Ctext%3AZapier%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A2425698%2Ctext%3ASegment%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A2497653%2Ctext%3ACrowdStrike%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A2499210%2Ctext%3AGusto%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A2652729%2Ctext%3ADrChrono%2520by%2520EverHealth%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A2659008%2Ctext%3ATray.ai%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A2684737%2Ctext%3APlaid%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A2696602%2Ctext%3ACodecademy%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A2705784%2Ctext%3AClever%2520Inc.%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A27137803%2Ctext%3AStarkWare%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A27139279%2Ctext%3ASendwave%2520%2528International%2520Remittance%2529%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A27159855%2Ctext%3AStarburst%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A2728700%2Ctext%3AAlgolia%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A2732417%2Ctext%3AInstacart%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A2760138%2Ctext%3AArctic%2520Wolf%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A27670348%2Ctext%3AMattermost%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A2783482%2Ctext%3ABird%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A2829210%2Ctext%3AAppZen%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A28405403%2Ctext%3ACelestia%2520Labs%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A28419091%2Ctext%3AWave%2520Mobile%2520Money%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A28491094%2Ctext%3An8n%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A2850862%2Ctext%3ACanva%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A285633%2Ctext%3AOMGPOP%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A2857634%2Ctext%3ACoinbase%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A28625893%2Ctext%3ABlameless%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A28657864%2Ctext%3ATines%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A29014871%2Ctext%3AAlchemy%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A29309454%2Ctext%3ALinear%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A30086%2Ctext%3APalo%2520Alto%2520Networks%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A3033313%2Ctext%3AStraiker%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A30581230%2Ctext%3ACortex%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A30846%2Ctext%3ASpaceX%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A3089380%2Ctext%3AAuth0%2520by%2520Okta%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A30898036%2Ctext%3ANotion%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A309694%2Ctext%3AAirbnb%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A3131483%2Ctext%3AFlexport%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A3157549%2Ctext%3AServiceTitan%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A3163250%2Ctext%3ACarta%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A3175085%2Ctext%3ABellabeat%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A3205573%2Ctext%3ADoorDash%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A3254263%2Ctext%3ARobinhood%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A33182031%2Ctext%3AFly.io%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A33460902%2Ctext%3AOffchain%2520Labs%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A3477522%2Ctext%3ADatabricks%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A3502352%2Ctext%3AWebflow%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A3509899%2Ctext%3ACircle%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A3524822%2Ctext%3ARigetti%2520Computing%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A3542130%2Ctext%3ADomino%2520Data%2520Lab%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A35462987%2Ctext%3AVanta%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A35478479%2Ctext%3AMiddesk%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A35496415%2Ctext%3ALever%2520VC%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A35528665%2Ctext%3ARedpanda%2520Data%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A35573984%2Ctext%3AOrca%2520Security%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A35625249%2Ctext%3ASketch%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A35653234%2Ctext%3ATailscale%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A3608%2Ctext%3ANVIDIA%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A36121341%2Ctext%3ACoreWeave%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A3640016%2Ctext%3ACalendly%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A3649864%2Ctext%3AGOAT%2520Group%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A3650502%2Ctext%3AFigma%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A3675685%2Ctext%3AWorkato%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A3738695%2Ctext%3AChime%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A3749939%2Ctext%3ALaunchDarkly%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A3767529%2Ctext%3ANubank%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A3769390%2Ctext%3ABenchling%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A3788927%2Ctext%3ARazorpay%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A3795851%2Ctext%3APostman%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A3954657%2Ctext%3AFivetran%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A3991822%2Ctext%3AAirtable%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A40057919%2Ctext%3AWorkOS%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A40654174%2Ctext%3AChronosphere%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A40671813%2Ctext%3ARobust%2520Intelligence%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A40708618%2Ctext%3AInjective%2520Labs%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A40769465%2Ctext%3AEthereum%2520Foundation%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A40773017%2Ctext%3AOdeko%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A42344868%2Ctext%3AHightouch%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A42692278%2Ctext%3AMetronome%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A42784537%2Ctext%3AZip%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A42785598%2Ctext%3AAleo%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A4803356%2Ctext%3ASourcegraph%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A4824711%2Ctext%3AGrubMarket%2520Inc.%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A4864798%2Ctext%3ANurx%2520from%2520Thirty%2520Madison%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A49644900%2Ctext%3ADragonfly%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A4997584%2Ctext%3AScentbird%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A5101804%2Ctext%3AGitLab%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A52192428%2Ctext%3APolkadot%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A5324440%2Ctext%3AProductboard%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A6392431%2Ctext%3ANetlify%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A6417671%2Ctext%3AClearbit%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A6424460%2Ctext%3ASentry%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A6442095%2Ctext%3ATripActions%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A6442270%2Ctext%3AMinIO%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A6455229%2Ctext%3ANotable%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A64890982%2Ctext%3AWiz%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A65281968%2Ctext%3ATecton%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A65574467%2Ctext%3ARailway%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A65604679%2Ctext%3AAva%2520Labs%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A65638805%2Ctext%3AMaterial%2520Security%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A6575553%2Ctext%3AByteDance%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A6578256%2Ctext%3ADremio%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A6589979%2Ctext%3APachyderm%2520Inc.%2520%2528Acquired%2520by%2520HPE%2529%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A66256146%2Ctext%3AMoxion%2520Power%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A66318622%2Ctext%3AStability%2520AI%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A6653417%2Ctext%3ASuperhuman%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A67081245%2Ctext%3ATemporal%2520Technologies%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A67094527%2Ctext%3ACerby%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A675562%2Ctext%3ASquare%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A68023390%2Ctext%3AIsland%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A68047275%2Ctext%3AUniswap%2520Labs%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A68302528%2Ctext%3ASnorkel%2520AI%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A69007278%2Ctext%3ADrata%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A69266957%2Ctext%3AAxelar%2520Network%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A69336355%2Ctext%3AClerk.com%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A69864141%2Ctext%3AVantage%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A70904927%2Ctext%3AJasper%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A70909767%2Ctext%3AFacebook%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A70975817%2Ctext%3AVarda%2520Space%2520Industries%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A71009477%2Ctext%3AFlutterFlow%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A71019539%2Ctext%3AStytch%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A71362299%2Ctext%3AAirplane%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A71528119%2Ctext%3ANEAR%2520Protocol%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A71534355%2Ctext%3ASurrealDB%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A71941732%2Ctext%3ARoam%2520Research%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A72016628%2Ctext%3AMomento%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A72287133%2Ctext%3ANeon%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A72621157%2Ctext%3ATessell%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A73802019%2Ctext%3ABaseten%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A74027874%2Ctext%3ALocalStack%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A74126343%2Ctext%3AAnthropic%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A74341323%2Ctext%3AOsmosis%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A74724401%2Ctext%3ALiveKit%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A748731%2Ctext%3AKlarna%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A74882602%2Ctext%3AGlean%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A74903143%2Ctext%3AReplicate%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A75003567%2Ctext%3AHadrian%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A75156029%2Ctext%3Aincident.io%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A75574122%2Ctext%3AFlashbots%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A75654012%2Ctext%3ALayerZero%2520Labs%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A7587729%2Ctext%3AIronclad%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A7588209%2Ctext%3AOklo%2520Inc%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A7602863%2Ctext%3AZipline%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A76174015%2Ctext%3AMysten%2520Labs%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A76213412%2Ctext%3AChainguard%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A76233021%2Ctext%3ADagger%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A76262108%2Ctext%3AKumo.AI%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A77101032%2Ctext%3ALanceDB%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A7792920%2Ctext%3AOpenInvest%252C%2520a%2520J.P.%2520Morgan%2520company%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A79119792%2Ctext%3AAptos%2520Labs%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A79149728%2Ctext%3AOmni%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A79159372%2Ctext%3AEspresso%2520Systems%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A80114151%2Ctext%3AClickHouse%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A80517752%2Ctext%3AMagic%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A81330326%2Ctext%3AAdept%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A81462746%2Ctext%3ABabylon%2520Labs%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A81491861%2Ctext%3APredibase%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A81739298%2Ctext%3AModular%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A81922043%2Ctext%3AMotherDuck%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A82157982%2Ctext%3AAltLayer%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A82318617%2Ctext%3AMidjourney%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A82340922%2Ctext%3AFixie.ai%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A82347897%2Ctext%3ASei%2520Labs%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A82546696%2Ctext%3Aunstructured.io%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A83019124%2Ctext%3AEigen%2520Labs%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A84802787%2Ctext%3AMonad%2520Labs%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A86583130%2Ctext%3AElevenLabs%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A86632358%2Ctext%3ALido%2520Finance%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A87215021%2Ctext%3ATigerBeetle%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A88026039%2Ctext%3ACodiumAI%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A88424181%2Ctext%3AResend%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A88873%2Ctext%3AConfluent%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A89266960%2Ctext%3ANorm%2520Ai%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A89486558%2Ctext%3ACharacter.AI%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A89566597%2Ctext%3AHarvey%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A89707979%2Ctext%3AEclipse%2520Labs%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A89905025%2Ctext%3AReflex%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A89962189%2Ctext%3AThe%2520Arbitrum%2520Foundation%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A89970028%2Ctext%3ASui%2520Foundation%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A90663044%2Ctext%3AChroma%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A90666371%2Ctext%3AZed%2520Industries%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A91113954%2Ctext%3AOptimism%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A91154103%2Ctext%3ALlamaIndex%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A91342320%2Ctext%3ALangChain%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A9230366%2Ctext%3ACopy.ai%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A9251256%2Ctext%3AEquipmentShare%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A926041%2Ctext%3AOkta%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A9309408%2Ctext%3ACockroach%2520Labs%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A93672648%2Ctext%3ADust%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A94273421%2Ctext%3AMistral%2520AI%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A94859563%2Ctext%3Apoolside%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A95057221%2Ctext%3Abitdrift%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A966527%2Ctext%3AGo1%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A98582116%2Ctext%3ARestate%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A98695525%2Ctext%3APatronus%2520AI%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))))%2C(type%3AINDUSTRY%2Cvalues%3AList((id%3A6%2Ctext%3ATechnology%252C%2520Information%2520and%2520Internet%2CselectionType%3AINCLUDED)%2C(id%3A4%2Ctext%3ASoftware%2520Development%2CselectionType%3AINCLUDED)%2C(id%3A43%2Ctext%3AFinancial%2520Services%2CselectionType%3AINCLUDED)%2C(id%3A1%2Ctext%3ADefense%2520and%2520Space%2520Manufacturing%2CselectionType%3AINCLUDED)%2C(id%3A114%2Ctext%3ANanotechnology%2520Research%2CselectionType%3AINCLUDED)%2C(id%3A118%2Ctext%3AComputer%2520and%2520Network%2520Security%2CselectionType%3AINCLUDED)%2C(id%3A12%2Ctext%3ABiotechnology%2520Research%2CselectionType%3AINCLUDED)%2C(id%3A15%2Ctext%3APharmaceutical%2520Manufacturing%2CselectionType%3AINCLUDED)%2C(id%3A2458%2Ctext%3AData%2520Infrastructure%2520and%2520Analytics%2CselectionType%3AINCLUDED)%2C(id%3A3089%2Ctext%3ASpace%2520Research%2520and%2520Technology%2CselectionType%3AINCLUDED)%2C(id%3A3099%2Ctext%3AEmbedded%2520Software%2520Products%2CselectionType%3AINCLUDED)%2C(id%3A3100%2Ctext%3AMobile%2520Computing%2520Software%2520Products%2CselectionType%3AINCLUDED)%2C(id%3A3101%2Ctext%3ADesktop%2520Computing%2520Software%2520Products%2CselectionType%3AINCLUDED)%2C(id%3A3102%2Ctext%3AIT%2520System%2520Custom%2520Software%2520Development%2CselectionType%3AINCLUDED)%2C(id%3A3106%2Ctext%3AIT%2520System%2520Data%2520Services%2CselectionType%3AINCLUDED)%2C(id%3A3107%2Ctext%3AIT%2520System%2520Testing%2520and%2520Evaluation%2CselectionType%3AINCLUDED)%2C(id%3A3130%2Ctext%3AData%2520Security%2520Software%2520Products%2CselectionType%3AINCLUDED)%2C(id%3A3134%2Ctext%3ABlockchain%2520Services%2CselectionType%3AINCLUDED)%2C(id%3A3242%2Ctext%3AEngineering%2520Services%2CselectionType%3AINCLUDED)%2C(id%3A3248%2Ctext%3ARobotics%2520Engineering%2CselectionType%3AINCLUDED)%2C(id%3A3252%2Ctext%3AClimate%2520Data%2520and%2520Analytics%2CselectionType%3AINCLUDED)%2C(id%3A5%2Ctext%3AComputer%2520Networking%2520Products%2CselectionType%3AINCLUDED)))%2C(type%3AREGION%2Cvalues%3AList((id%3A102221843%2Ctext%3ANorth%2520America%2CselectionType%3AINCLUDED)%2C(id%3A103644278%2Ctext%3AUnited%2520States%2CselectionType%3AINCLUDED)%2C(id%3A104514572%2Ctext%3ASouth%2520America%2CselectionType%3AINCLUDED)%2C(id%3A100506914%2Ctext%3AEurope%2CselectionType%3AINCLUDED)%2C(id%3A101174742%2Ctext%3ACanada%2CselectionType%3AINCLUDED)))%2C(type%3AFUNCTION%2Cvalues%3AList((id%3A13%2Ctext%3AInformation%2520Technology%2CselectionType%3AINCLUDED)%2C(id%3A18%2Ctext%3AOperations%2CselectionType%3AINCLUDED)%2C(id%3A25%2Ctext%3ASales%2CselectionType%3AINCLUDED)%2C(id%3A4%2Ctext%3ABusiness%2520Development%2CselectionType%3AINCLUDED)%2C(id%3A10%2Ctext%3AFinance%2CselectionType%3AINCLUDED)%2C(id%3A8%2Ctext%3AEngineering%2CselectionType%3AINCLUDED)%2C(id%3A9%2Ctext%3AEntrepreneurship%2CselectionType%3AINCLUDED)%2C(id%3A19%2Ctext%3AProduct%2520Management%2CselectionType%3AINCLUDED)%2C(id%3A24%2Ctext%3AResearch%2CselectionType%3AINCLUDED)%2C(id%3A15%2Ctext%3AMarketing%2CselectionType%3AINCLUDED)%2C(id%3A11%2Ctext%3AHealthcare%2520Services%2CselectionType%3AINCLUDED)%2C(id%3A17%2Ctext%3AMilitary%2520and%2520Protective%2520Services%2CselectionType%3AINCLUDED)))%2C(type%3ACOMPANY_HEADQUARTERS%2Cvalues%3AList((id%3A102221843%2Ctext%3ANorth%2520America%2CselectionType%3AINCLUDED)%2C(id%3A103644278%2Ctext%3AUnited%2520States%2CselectionType%3AINCLUDED)%2C(id%3A100506914%2Ctext%3AEurope%2CselectionType%3AINCLUDED)%2C(id%3A101174742%2Ctext%3ACanada%2CselectionType%3AINCLUDED)%2C(id%3A104514572%2Ctext%3ASouth%2520America%2CselectionType%3AINCLUDED)))%2C(type%3ACOMPANY_TYPE%2Cvalues%3AList((id%3AP%2Ctext%3APrivately%2520Held%2CselectionType%3AINCLUDED)%2C(id%3AC%2Ctext%3APublic%2520Company%2CselectionType%3AINCLUDED)))%2C(type%3AYEARS_AT_CURRENT_COMPANY%2Cvalues%3AList((id%3A1%2Ctext%3ALess%2520than%25201%2520year%2CselectionType%3AINCLUDED)))%2C(type%3AYEARS_IN_CURRENT_POSITION%2Cvalues%3AList((id%3A1%2Ctext%3ALess%2520than%25201%2520year%2CselectionType%3AINCLUDED)))%2C(type%3ARELATIONSHIP%2Cvalues%3AList((id%3AF%2Ctext%3A1st%2520degree%2520connections%2CselectionType%3AINCLUDED)%2C(id%3AS%2Ctext%3A2nd%2520degree%2520connections%2CselectionType%3AINCLUDED)%2C(id%3AO%2Ctext%3A3rd%252B%2520degree%2520connections%2CselectionType%3AINCLUDED)))%2C(type%3ALEAD_INTERACTIONS%2Cvalues%3AList((id%3ALIMP%2Ctext%3AMessaged%2CselectionType%3AEXCLUDED))))%2Ckeywords%3A%2522something%2520new%2522%2520OR%2520%2522stealth%2522)&sessionId=jFoOWCqiTl2JVJ3QPZrL4g%3D%3D
"""

### Part 1: Sales Nav Scraper

In [ ]:
# Sales Nav Scraping
driver = instantiate_driver()
already_scraped_urls, to_scrape_urls = check_and_load_pickles(historyPickle, toScrapePickle)
to_scrape_urls = scrape_profiles(driver, SALES_NAV_SEARCH_URL, already_scraped_urls, to_scrape_urls, toScrapePickle)
print(str(len(to_scrape_urls)) + "profiles identified for scraping") 

### Part 2: Profile scraper, export data to Google Sheets, and reset results pickle

In [ ]:
# Profile Scraping
deduplicate(toScrapePickle, to_scrape_urls, already_scraped_urls)
results = scrape_all_profiles(driver, historyPickle, toScrapePickle, resultsPickle, failedURLsTextFile)


# Export results to Google Sheets
worksheet_title = 'Main'
resultsPickle = 'db/main_results.pickle'
results = readPickle(resultsPickle)
if results:
    worksheet_url = export_sheet(results, spreadsheet, worksheet_title)
    print(f"Worksheet updated: {worksheet_url}")
else:
    print("No data to export.")


# Reset results
results = []
writePickle(resultsPickle, results)
driver.close()

# QUERY 2: INFRA COMPANIES

### Setup

In [ ]:
toScrapePickle = 'db/infra_to_scrape.pickle'
resultsPickle = 'db/infra_results.pickle'
failedURLsTextFile = 'db/infra_failed_urls.txt'
historyPickle = 'db/history.pickle'

SALES_NAV_SEARCH_URL = """
https://www.linkedin.com/sales/search/people#query=(spellCorrectionEnabled%3Atrue%2CrecentSearchParam%3A(id%3A3934788460%2CdoLogHistory%3Atrue)%2Cfilters%3AList((type%3ACURRENT_COMPANY%2Cvalues%3AList((id%3Aurn%253Ali%253Aorganization%253A91313799%2Ctext%3AStealth%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A18583501%2Ctext%3AStealth%2520Startup%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A18016269%2Ctext%3AStealth%2520Mode%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A96670793%2Ctext%3AStealth%2520AI%2520Startup%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A31285650%2Ctext%3ANewCo%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A91194134%2Ctext%3ASomething%2520new%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))))%2C(type%3APAST_COMPANY%2Cvalues%3AList((text%3ATruEra%2CselectionType%3AINCLUDED)%2C(text%3ADagster%2520Labs%2CselectionType%3AINCLUDED)%2C(text%3APrefect%2CselectionType%3AINCLUDED)%2C(text%3ASuperconductive%2520Components%2520Inc%2CselectionType%3AINCLUDED)%2C(text%3Avectorized.io%2CselectionType%3AINCLUDED)%2C(text%3AKinesis%2520Money%2CselectionType%3AINCLUDED)%2C(text%3AArthur%2CselectionType%3AINCLUDED)%2C(text%3ADeepnote%2CselectionType%3AINCLUDED)%2C(text%3AScaleAI%2CselectionType%3AINCLUDED)%2C(text%3AMetaphor%2520Data%2CselectionType%3AINCLUDED)%2C(text%3APlanetScale%2CselectionType%3AINCLUDED)%2C(text%3AFastly%2CselectionType%3AINCLUDED)%2C(text%3AEdge%2520Delta%2CselectionType%3AINCLUDED)%2C(text%3AGrafana%2520Labs%2CselectionType%3AINCLUDED)%2C(text%3ADynatrace%2CselectionType%3AINCLUDED)%2C(text%3ACape%2CselectionType%3AINCLUDED)%2C(text%3APanther%2CselectionType%3AINCLUDED)%2C(text%3AArize%2520AI%2CselectionType%3AINCLUDED)%2C(text%3ACalypsoAI%2CselectionType%3AINCLUDED)%2C(text%3AProtect%2520AI%2CselectionType%3AINCLUDED)%2C(text%3AFiddler%2520AI%2CselectionType%3AINCLUDED)%2C(text%3AMONA%2520AI%2CselectionType%3AINCLUDED)%2C(text%3ACredo%2520AI%2CselectionType%3AINCLUDED)%2C(text%3ALakera%2CselectionType%3AINCLUDED)%2C(text%3ATonic.ai%2CselectionType%3AINCLUDED)%2C(text%3AMOSTLY%2520AI%2CselectionType%3AINCLUDED)%2C(text%3AGretel%2CselectionType%3AINCLUDED)%2C(text%3ALandingAI%2CselectionType%3AINCLUDED)%2C(text%3AWeights%2520%2526%2520Biases%2CselectionType%3AINCLUDED)%2C(text%3AOctoAI%2CselectionType%3AINCLUDED)%2C(text%3ALabelbox%2CselectionType%3AINCLUDED)%2C(text%3AContinual%2CselectionType%3AINCLUDED)%2C(text%3ADomino%2520Data%2520Lab%2CselectionType%3AINCLUDED)%2C(text%3AOuterbounds%2CselectionType%3AINCLUDED)%2C(text%3ACyberArk%2CselectionType%3AINCLUDED)%2C(text%3ASymantec%2CselectionType%3AINCLUDED)%2C(text%3AMcAfee%2CselectionType%3AINCLUDED)%2C(text%3ARapid7%2CselectionType%3AINCLUDED)%2C(text%3AQualys%2CselectionType%3AINCLUDED)%2C(text%3AVaronis%2CselectionType%3AINCLUDED)%2C(text%3AFortinet%2CselectionType%3AINCLUDED)%2C(text%3AXM%2520Cyber%2CselectionType%3AINCLUDED)%2C(text%3ADuo%2520Security%2CselectionType%3AINCLUDED)%2C(text%3ABetterCloud%2CselectionType%3AINCLUDED)%2C(text%3ASecurityScorecard%2CselectionType%3AINCLUDED)))%2C(type%3AINDUSTRY%2Cvalues%3AList((id%3A10%2Ctext%3ALegal%2520Services%2CselectionType%3AEXCLUDED)%2C(id%3A1080%2Ctext%3AHousehold%2520and%2520Institutional%2520Furniture%2520Manufacturing%2CselectionType%3AEXCLUDED)%2C(id%3A1095%2Ctext%3AMattress%2520and%2520Blinds%2520Manufacturing%2CselectionType%3AEXCLUDED)%2C(id%3A16%2Ctext%3AVeterinary%2520Services%2CselectionType%3AEXCLUDED)%2C(id%3A19%2Ctext%3ARetail%2520Apparel%2520and%2520Fashion%2CselectionType%3AEXCLUDED)%2C(id%3A20%2Ctext%3ASporting%2520Goods%2520Manufacturing%2CselectionType%3AEXCLUDED)%2C(id%3A2050%2Ctext%3AOptometrists%2CselectionType%3AEXCLUDED)%2C(id%3A2054%2Ctext%3APhysical%252C%2520Occupational%2520and%2520Speech%2520Therapists%2CselectionType%3AEXCLUDED)%2C(id%3A21%2Ctext%3ATobacco%2520Manufacturing%2CselectionType%3AEXCLUDED)%2C(id%3A3126%2Ctext%3AInterior%2520Design%2CselectionType%3AEXCLUDED)%2C(id%3A3132%2Ctext%3AInternet%2520Publishing%2CselectionType%3AEXCLUDED)%2C(id%3A57%2Ctext%3AOil%2520and%2520Gas%2CselectionType%3AEXCLUDED)%2C(id%3A61%2Ctext%3APaper%2520and%2520Forest%2520Product%2520Manufacturing%2CselectionType%3AEXCLUDED)%2C(id%3A62%2Ctext%3ARailroad%2520Equipment%2520Manufacturing%2CselectionType%3AEXCLUDED)%2C(id%3A64%2Ctext%3ARanching%2CselectionType%3AEXCLUDED)%2C(id%3A66%2Ctext%3AFisheries%2CselectionType%3AEXCLUDED)%2C(id%3A7%2Ctext%3ASemiconductor%2520Manufacturing%2CselectionType%3AEXCLUDED)%2C(id%3A71%2Ctext%3AArmed%2520Forces%2CselectionType%3AEXCLUDED)%2C(id%3A72%2Ctext%3ALegislative%2520Offices%2CselectionType%3AEXCLUDED)%2C(id%3A8%2Ctext%3ATelecommunications%2CselectionType%3AEXCLUDED)%2C(id%3A9%2Ctext%3ALaw%2520Practice%2CselectionType%3AEXCLUDED)))%2C(type%3AREGION%2Cvalues%3AList((id%3A103644278%2Ctext%3AUnited%2520States%2CselectionType%3AINCLUDED)%2C(id%3A100506914%2Ctext%3AEurope%2CselectionType%3AINCLUDED)%2C(id%3A101165590%2Ctext%3AUnited%2520Kingdom%2CselectionType%3AINCLUDED)%2C(id%3A101620260%2Ctext%3AIsrael%2CselectionType%3AINCLUDED)%2C(id%3A101174742%2Ctext%3ACanada%2CselectionType%3AINCLUDED)%2C(id%3A102713980%2Ctext%3AIndia%2CselectionType%3AEXCLUDED)%2C(id%3A103537801%2Ctext%3AAfrica%2CselectionType%3AEXCLUDED)%2C(id%3A104305776%2Ctext%3AUnited%2520Arab%2520Emirates%2CselectionType%3AEXCLUDED)))%2C(type%3AFUNCTION%2Cvalues%3AList((id%3A1%2Ctext%3AAccounting%2CselectionType%3AEXCLUDED)%2C(id%3A12%2Ctext%3AHuman%2520Resources%2CselectionType%3AEXCLUDED)%2C(id%3A15%2Ctext%3AMarketing%2CselectionType%3AEXCLUDED)%2C(id%3A2%2Ctext%3AAdministrative%2CselectionType%3AEXCLUDED)%2C(id%3A26%2Ctext%3ACustomer%2520Success%2520and%2520Support%2CselectionType%3AEXCLUDED)%2C(id%3A3%2Ctext%3AArts%2520and%2520Design%2CselectionType%3AEXCLUDED)))%2C(type%3AYEARS_IN_CURRENT_POSITION%2Cvalues%3AList((id%3A1%2Ctext%3ALess%2520than%25201%2520year%2CselectionType%3AINCLUDED)))%2C(type%3APROFILE_LANGUAGE%2Cvalues%3AList((id%3Aen%2Ctext%3AEnglish%2CselectionType%3AINCLUDED)))%2C(type%3ALEAD_INTERACTIONS%2Cvalues%3AList((id%3ALIMP%2Ctext%3AMessaged%2CselectionType%3AEXCLUDED)))%2C(type%3ACURRENT_TITLE%2Cvalues%3AList((id%3A35%2Ctext%3AFounder%2CselectionType%3AINCLUDED)%2C(id%3A103%2Ctext%3ACo-Founder%2CselectionType%3AINCLUDED)%2C(id%3A8%2Ctext%3AChief%2520Executive%2520Officer%2CselectionType%3AINCLUDED)%2C(id%3A153%2Ctext%3AChief%2520Technology%2520Officer%2CselectionType%3AINCLUDED))))%2Ckeywords%3A%2522something%2520new%2522%2520OR%2520%2522stealth%2522)&sessionId=R7vv8AiYRXiqMseY1m3CwQ%3D%3D
"""

### Part 1: Sales Nav Scraper

In [ ]:
# Sales Nav Scraping
driver = instantiate_driver()
already_scraped_urls, to_scrape_urls = check_and_load_pickles(historyPickle, toScrapePickle)
to_scrape_urls = scrape_profiles(driver, SALES_NAV_SEARCH_URL, already_scraped_urls, to_scrape_urls, toScrapePickle)
print(str(len(to_scrape_urls)) + "profiles identified for scraping") 

### Part 2: Profile scraper, export data to Google Sheets, and reset results pickle

In [ ]:
# Profile Scraping
deduplicate(toScrapePickle, to_scrape_urls, already_scraped_urls)
results = scrape_all_profiles(driver, historyPickle, toScrapePickle, resultsPickle, failedURLsTextFile)


# Export results to Google Sheets
worksheet_title = 'Infra'
resultsPickle = 'db/infra_results.pickle'
results = readPickle(resultsPickle)
if results:
    worksheet_url = export_sheet(results, spreadsheet, worksheet_title)
    print(f"Worksheet updated: {worksheet_url}")
else:
    print("No data to export.")


# Reset results
results = []
writePickle(resultsPickle, results)
driver.close()

# QUERY 3: VC PORTFOLIO COMPANIES

### Setup

In [ ]:
toScrapePickle = 'db/portfolio_to_scrape.pickle'
resultsPickle = 'db/portfolio_results.pickle'
failedURLsTextFile = 'db/portfolio_failed_urls.txt'
historyPickle = 'db/history.pickle'

SALES_NAV_SEARCH_URL = """
https://www.linkedin.com/sales/search/people#query=(spellCorrectionEnabled%3Atrue%2CrecentSearchParam%3A(id%3A3934788452%2CdoLogHistory%3Atrue)%2Cfilters%3AList((type%3ACURRENT_COMPANY%2Cvalues%3AList((id%3Aurn%253Ali%253Aorganization%253A91313799%2Ctext%3AStealth%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A18583501%2Ctext%3AStealth%2520Startup%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A96670793%2Ctext%3AStealth%2520AI%2520Startup%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A18016269%2Ctext%3AStealth%2520Mode%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A99152379%2Ctext%3AStealth%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A31285650%2Ctext%3ANewCo%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))))%2C(type%3APAST_COMPANY%2Cvalues%3AList((text%3AKlaviyo%2CselectionType%3AINCLUDED)%2C(text%3ADust%2CselectionType%3AINCLUDED)%2C(text%3AEthos%2520Life%2520Agency%2CselectionType%3AINCLUDED)%2C(text%3AFactory%2CselectionType%3AINCLUDED)%2C(text%3AQuantum%2520Circuits%252C%2520Inc.%2CselectionType%3AINCLUDED)%2C(text%3AxAI%2CselectionType%3AINCLUDED)%2C(text%3ANeros%2520Technologies%2CselectionType%3AINCLUDED)%2C(text%3AAlkira%252C%2520Inc.%2CselectionType%3AINCLUDED)%2C(text%3AAPEX%2CselectionType%3AINCLUDED)%2C(text%3AStrongDM%2CselectionType%3AINCLUDED)%2C(text%3AOasis%2520Security%2CselectionType%3AINCLUDED)%2C(text%3AAnrok%2CselectionType%3AINCLUDED)%2C(text%3ACollaborative%2520Robotics%2CselectionType%3AINCLUDED)%2C(text%3AGrow%2520Therapy%2CselectionType%3AINCLUDED)%2C(text%3AZafran%2520Security%2CselectionType%3AINCLUDED)%2C(text%3AFireworks%2520AI%2CselectionType%3AINCLUDED)%2C(text%3AFoundry%2CselectionType%3AINCLUDED)%2C(text%3APhysical%2520Intelligence%2CselectionType%3AINCLUDED)%2C(text%3ARobCo%2CselectionType%3AINCLUDED)%2C(text%3AMeter%2CselectionType%3AINCLUDED)%2C(text%3APennylane%2CselectionType%3AINCLUDED)%2C(text%3AGuardant%2520Health%2CselectionType%3AINCLUDED)%2C(text%3ATacto%2CselectionType%3AINCLUDED)%2C(text%3APrivy%2CselectionType%3AINCLUDED)%2C(text%3AApollo.io%2CselectionType%3AINCLUDED)%2C(text%3ARelease%2CselectionType%3AINCLUDED)%2C(text%3ACaptions%2CselectionType%3AINCLUDED)%2C(text%3AWarp%2CselectionType%3AINCLUDED)%2C(text%3AMach%2520Industries%2CselectionType%3AINCLUDED)%2C(text%3AMedallion%2CselectionType%3AINCLUDED)%2C(text%3ADeno%2CselectionType%3AINCLUDED)%2C(text%3ACybersyn%2CselectionType%3AINCLUDED)%2C(text%3ACaldera%2CselectionType%3AINCLUDED)%2C(text%3ATavus%2CselectionType%3AINCLUDED)%2C(text%3ADagster%2520Labs%2CselectionType%3AINCLUDED)%2C(text%3ABigeye%2CselectionType%3AINCLUDED)%2C(text%3AMutiny%2CselectionType%3AINCLUDED)%2C(text%3AKnowde%2CselectionType%3AINCLUDED)%2C(text%3AStatsig%2CselectionType%3AINCLUDED)%2C(text%3APhysna%2CselectionType%3AINCLUDED)%2C(text%3AStreamlit%2CselectionType%3AINCLUDED)%2C(text%3ANeeva%2CselectionType%3AINCLUDED)%2C(text%3ALoom%2CselectionType%3AINCLUDED)%2C(text%3AAurora%2CselectionType%3AINCLUDED)%2C(text%3ATessian%2CselectionType%3AINCLUDED)%2C(text%3AAmplitude%2CselectionType%3AINCLUDED)%2C(text%3AAthelas%2CselectionType%3AINCLUDED)%2C(text%3AFilecoin%2520Labs%2CselectionType%3AINCLUDED)%2C(text%3AUnity%2CselectionType%3AINCLUDED)%2C(text%3AWhatsApp%2CselectionType%3AINCLUDED)%2C(text%3ACisco%2520Meraki%2CselectionType%3AINCLUDED)%2C(text%3ABlock%2CselectionType%3AINCLUDED)%2C(text%3ACisco%2520ThousandEyes%2CselectionType%3AINCLUDED)%2C(text%3ANimble%2520Storage%252C%2520acquired%2520by%2520Hewlett%2520Packard%2520Enterprise%2520company%2520in%25202017%2CselectionType%3AINCLUDED)%2C(text%3AFireEye%252C%2520Inc.%2CselectionType%3AINCLUDED)%2C(text%3AUpwind%2520Security%2CselectionType%3AINCLUDED)%2C(text%3AOnehouse%2CselectionType%3AINCLUDED)%2C(text%3ASeven%2520AI%2CselectionType%3AINCLUDED)%2C(text%3ABedrock%2520Security%2CselectionType%3AINCLUDED)%2C(text%3AWarpStream%2CselectionType%3AINCLUDED)%2C(text%3AKodem%2CselectionType%3AINCLUDED)%2C(text%3ABiogenesis%2CselectionType%3AINCLUDED)%2C(text%3ABraintrust%2CselectionType%3AINCLUDED)%2C(text%3AOpal%2520Security%2CselectionType%3AINCLUDED)%2C(text%3AGreenlite%2CselectionType%3AINCLUDED)%2C(text%3ACensys%2CselectionType%3AINCLUDED)%2C(text%3AApiiro%2CselectionType%3AINCLUDED)%2C(text%3ATome%2CselectionType%3AINCLUDED)%2C(text%3AStackBlitz%2CselectionType%3AINCLUDED)%2C(text%3AObsidian%2520Security%2CselectionType%3AINCLUDED)%2C(text%3AMarqeta%2CselectionType%3AINCLUDED)%2C(text%3AAppDynamics%2CselectionType%3AINCLUDED)%2C(text%3ASprig%253A%2520Eat%2520Well%2CselectionType%3AINCLUDED)%2C(text%3AOpenDNS%2CselectionType%3AINCLUDED)%2C(text%3APure%2520Storage%2CselectionType%3AINCLUDED)%2C(text%3AIBM%2520Instana%2CselectionType%3AINCLUDED)%2C(text%3ATrace%2520Data%2520%2528acquired%2520by%2520Netskope%2529%2CselectionType%3AINCLUDED)%2C(text%3ARaycast%2CselectionType%3AINCLUDED)%2C(text%3ACyCognito%2CselectionType%3AINCLUDED)%2C(text%3ADeliveroo%2CselectionType%3AINCLUDED)%2C(text%3ASpotify%2CselectionType%3AINCLUDED)%2C(text%3AAtlassian%2CselectionType%3AINCLUDED)%2C(text%3AHopin%2CselectionType%3AINCLUDED)%2C(text%3ATranscend%2CselectionType%3AINCLUDED)%2C(text%3AFramer%2CselectionType%3AINCLUDED)%2C(text%3AMonzo%2520Bank%2CselectionType%3AINCLUDED)%2C(text%3ASlack%2CselectionType%3AINCLUDED)%2C(text%3AAssemblyAI%2CselectionType%3AINCLUDED)%2C(text%3AProjectDiscovery%2CselectionType%3AINCLUDED)%2C(text%3ACatch%252BRelease%2CselectionType%3AINCLUDED)%2C(text%3ASprinter%2520Health%2CselectionType%3AINCLUDED)%2C(text%3AXendit%2CselectionType%3AINCLUDED)%2C(text%3APoggio%2CselectionType%3AINCLUDED)%2C(text%3ATonkean%2CselectionType%3AINCLUDED)%2C(text%3AFootprint%2CselectionType%3AINCLUDED)%2C(text%3AAugment%2CselectionType%3AINCLUDED)%2C(text%3AVizcom%2CselectionType%3AINCLUDED)%2C(text%3ASuperlinked%2CselectionType%3AINCLUDED)%2C(text%3AAdaptive%2520ML%2CselectionType%3AINCLUDED)%2C(text%3AIdeogram%2CselectionType%3AINCLUDED)%2C(text%3AWeaviate%2CselectionType%3AINCLUDED)%2C(text%3ATwelve%2520Labs%2CselectionType%3AINCLUDED)%2C(text%3AHebbia%2CselectionType%3AINCLUDED)%2C(text%3ANewfront%2CselectionType%3AINCLUDED)%2C(text%3AShanghai%2520Superconductive%2520Sensor%2520Factory%2CselectionType%3AINCLUDED)%2C(text%3AGreat%2520Expectations%2CselectionType%3AINCLUDED)%2C(text%3ARoblox%2CselectionType%3AINCLUDED)%2C(text%3AFleetsmith%2CselectionType%3AINCLUDED)%2C(text%3ARevenueCat%2CselectionType%3AINCLUDED)%2C(text%3ALithic%2CselectionType%3AINCLUDED)%2C(text%3ACovariant%2CselectionType%3AINCLUDED)%2C(text%3AHippocratic%2520AI%2CselectionType%3AINCLUDED)%2C(text%3ALuma%2520AI%2CselectionType%3AINCLUDED)%2C(text%3AFinch%2CselectionType%3AINCLUDED)%2C(text%3AVial%2CselectionType%3AINCLUDED)%2C(text%3ARibbon%2520Health%2CselectionType%3AINCLUDED)%2C(text%3AAuthZed%2CselectionType%3AINCLUDED)%2C(text%3AVannevar%2520Labs%2CselectionType%3AINCLUDED)%2C(text%3Aadonis%2CselectionType%3AINCLUDED)%2C(text%3AAtoB%2CselectionType%3AINCLUDED)%2C(text%3AGitpod%2CselectionType%3AINCLUDED)%2C(text%3APolymarket%2CselectionType%3AINCLUDED)%2C(text%3ANominal%2CselectionType%3AINCLUDED)%2C(text%3ASymbolica%2520AI%2CselectionType%3AINCLUDED)%2C(text%3ASnap%2520Inc.%2CselectionType%3AINCLUDED)%2C(text%3AAnduril%2520Industries%2CselectionType%3AINCLUDED)%2C(text%3ANova%2520Credit%2CselectionType%3AINCLUDED)%2C(text%3AImpira%2520%2528acquired%2520by%2520Figma%2529%2CselectionType%3AINCLUDED)%2C(text%3AOscar%2520Health%2CselectionType%3AINCLUDED)%2C(text%3ACrusoe%2CselectionType%3AINCLUDED)%2C(text%3AAkash%2520Systems%2CselectionType%3AINCLUDED)%2C(text%3AArmada%2CselectionType%3AINCLUDED)%2C(text%3ASynthego%2520Corporation%2CselectionType%3AINCLUDED)%2C(text%3ATraba%2CselectionType%3AINCLUDED)%2C(text%3ATruckSmarter%2CselectionType%3AINCLUDED)%2C(text%3AAffirm%2CselectionType%3AINCLUDED)%2C(text%3APalantir%2520Technologies%2CselectionType%3AINCLUDED)%2C(text%3ATwilio%2CselectionType%3AINCLUDED)%2C(text%3AAvail%2CselectionType%3AINCLUDED)%2C(text%3ANorthwood%2CselectionType%3AINCLUDED)%2C(text%3ADynamic%2CselectionType%3AINCLUDED)%2C(text%3ALayer%2520N%2CselectionType%3AINCLUDED)%2C(text%3AGrit%2CselectionType%3AINCLUDED)%2C(text%3ANeuralink%2CselectionType%3AINCLUDED)%2C(text%3AHalcyon%2CselectionType%3AINCLUDED)%2C(text%3ABuf%2CselectionType%3AINCLUDED)%2C(text%3ARecursion%2CselectionType%3AINCLUDED)%2C(text%3ASaildrone%2CselectionType%3AINCLUDED)%2C(text%3ADesktop%2520Metal%2CselectionType%3AINCLUDED)%2C(text%3AEvolutionaryScale%2CselectionType%3AINCLUDED)%2C(text%3AFormic%2CselectionType%3AINCLUDED)%2C(text%3ALayup%2520Parts%2CselectionType%3AINCLUDED)%2C(text%3AImpulse%2520Space%2CselectionType%3AINCLUDED)%2C(text%3ACoder%2CselectionType%3AINCLUDED)%2C(text%3AOwner.com%2CselectionType%3AINCLUDED)%2C(text%3AAttio%2CselectionType%3AINCLUDED)%2C(text%3AFixie.ai%2CselectionType%3AINCLUDED)%2C(text%3AKustomer%2CselectionType%3AINCLUDED)%2C(text%3ADragonflyDB%2CselectionType%3AINCLUDED)%2C(text%3ALILT%2CselectionType%3AINCLUDED)%2C(text%3AGuild%2CselectionType%3AINCLUDED)%2C(text%3ADuo%2520Security%2CselectionType%3AINCLUDED)%2C(text%3AHeroku%2CselectionType%3AINCLUDED)%2C(text%3ADecagon%2CselectionType%3AINCLUDED)%2C(text%3AInngest%2CselectionType%3AINCLUDED)%2C(text%3ATurquoise%2520Health%2CselectionType%3AINCLUDED)%2C(text%3AMatik%2CselectionType%3AINCLUDED)%2C(text%3ARUNWAY%2520%25E2%2580%2593%2520A%2520Financial%2520Innovation%2520Firm%2CselectionType%3AINCLUDED)%2C(text%3AAztec%2CselectionType%3AINCLUDED)%2C(text%3ATruffle%2520Security%2520Co.%2CselectionType%3AINCLUDED)%2C(text%3AMemora%2520Health%2CselectionType%3AINCLUDED)%2C(text%3AFoursquare%2CselectionType%3AINCLUDED)%2C(text%3ALumos%2CselectionType%3AINCLUDED)%2C(text%3AGraphite%2CselectionType%3AINCLUDED)%2C(text%3AConvex%2CselectionType%3AINCLUDED)%2C(text%3ATackle.io%2CselectionType%3AINCLUDED)%2C(text%3ANansen%2CselectionType%3AINCLUDED)%2C(text%3ATomorrow%2520Health%2CselectionType%3AINCLUDED)%2C(text%3APlanetScale%2CselectionType%3AINCLUDED)%2C(text%3ASentiLink%2CselectionType%3AINCLUDED)%2C(text%3ASisu%2CselectionType%3AINCLUDED)%2C(text%3ACommerce%2520Layer%2CselectionType%3AINCLUDED)%2C(text%3AGlide%2CselectionType%3AINCLUDED)%2C(text%3AHackerOne%2CselectionType%3AINCLUDED)%2C(text%3APomerium%2CselectionType%3AINCLUDED)%2C(text%3ALeya%2CselectionType%3AINCLUDED)%2C(text%3AHeyGen%2CselectionType%3AINCLUDED)%2C(text%3ABezi%2CselectionType%3AINCLUDED)%2C(text%3AMindsDB%2CselectionType%3AINCLUDED)%2C(text%3ALevelpath%2CselectionType%3AINCLUDED)%2C(text%3ACodeium%2CselectionType%3AINCLUDED)%2C(text%3AHandshake%2CselectionType%3AINCLUDED)%2C(text%3ASuperblocks%2CselectionType%3AINCLUDED)%2C(text%3ARudderStack%2CselectionType%3AINCLUDED)%2C(text%3AViz.ai%2CselectionType%3AINCLUDED)%2C(text%3ASpinLaunch%2CselectionType%3AINCLUDED)%2C(text%3AWindBorne%2520Systems%2CselectionType%3AINCLUDED)%2C(text%3ASelect%2520Star%2CselectionType%3AINCLUDED)%2C(text%3ADexterity%252C%2520Inc.%2CselectionType%3AINCLUDED)%2C(text%3AMuleSoft%2CselectionType%3AINCLUDED)%2C(text%3ANutanix%2CselectionType%3AINCLUDED)%2C(text%3ASubstrate%2CselectionType%3AINCLUDED)%2C(text%3ACypress.io%2CselectionType%3AINCLUDED)%2C(text%3APipedrive%2CselectionType%3AINCLUDED)%2C(text%3ADashlane%2CselectionType%3AINCLUDED)%2C(text%3APinterest%2CselectionType%3AINCLUDED)%2C(text%3AMerge%2CselectionType%3AINCLUDED)%2C(text%3ASakana%2520AI%2CselectionType%3AINCLUDED)%2C(text%3AStrapi%2CselectionType%3AINCLUDED)%2C(text%3APipedream%2CselectionType%3AINCLUDED)%2C(text%3ASleuth%2CselectionType%3AINCLUDED)%2C(text%3AFleet%2520Device%2520Management%2CselectionType%3AINCLUDED)%2C(text%3ALegit%2520Security%2CselectionType%3AINCLUDED)%2C(text%3ATinybird%2CselectionType%3AINCLUDED)%2C(text%3AApex%2520-%2520Spacecraft%2520Manufacturing%2CselectionType%3AINCLUDED)%2C(text%3AMeilisearch%2CselectionType%3AINCLUDED)%2C(text%3AWorkstream%2CselectionType%3AINCLUDED)%2C(text%3AStarTree%2CselectionType%3AINCLUDED)%2C(text%3ATorq%2CselectionType%3AINCLUDED)%2C(text%3AEssential%2520AI%2CselectionType%3AINCLUDED)%2C(text%3ACodegen%2CselectionType%3AINCLUDED)%2C(text%3ACharm%2520Industrial%2CselectionType%3AINCLUDED)%2C(text%3AFabric%2CselectionType%3AINCLUDED)%2C(text%3ACommandBar%2CselectionType%3AINCLUDED)%2C(text%3AParafin%2CselectionType%3AINCLUDED)%2C(text%3APitch%2CselectionType%3AINCLUDED)%2C(text%3ASaronic%2520Technologies%2CselectionType%3AINCLUDED)%2C(text%3AIntegral%2CselectionType%3AINCLUDED)%2C(text%3ALatchBio%2CselectionType%3AINCLUDED)%2C(text%3ABitwise%2520Asset%2520Management%2CselectionType%3AINCLUDED)%2C(text%3ACapchase%2CselectionType%3AINCLUDED)%2C(text%3ABioAge%2520Labs%2CselectionType%3AINCLUDED)%2C(text%3ACloudTrucks%2CselectionType%3AINCLUDED)%2C(text%3AAshby%2CselectionType%3AINCLUDED)%2C(text%3ADatologyAI%2CselectionType%3AINCLUDED)%2C(text%3ALamini%2CselectionType%3AINCLUDED)%2C(text%3AScribe%2CselectionType%3AINCLUDED)%2C(text%3AAntithesis%2CselectionType%3AINCLUDED)%2C(text%3AMotif%2CselectionType%3AINCLUDED)%2C(text%3Atldraw%2CselectionType%3AINCLUDED)%2C(text%3AFastly%2CselectionType%3AINCLUDED)%2C(text%3AParrot%2CselectionType%3AINCLUDED)%2C(text%3APostgresML%2CselectionType%3AINCLUDED)%2C(text%3AMetaphor%2520Data%2CselectionType%3AINCLUDED)%2C(text%3ACubist%2CselectionType%3AINCLUDED)%2C(text%3ADiagrid%2CselectionType%3AINCLUDED)%2C(text%3AEppo%2CselectionType%3AINCLUDED)%2C(text%3ATrustdome%2520%2528acquired%2520by%2520Zscaler%2529%2CselectionType%3AINCLUDED)%2C(text%3AHelion%2CselectionType%3AINCLUDED)%2C(text%3ABrowserbase%2CselectionType%3AINCLUDED)%2C(text%3ASanity%2CselectionType%3AINCLUDED)%2C(text%3AKnock%2CselectionType%3AINCLUDED)%2C(text%3ADexa%2CselectionType%3AINCLUDED)%2C(text%3ASpline%2CselectionType%3AINCLUDED)%2C(text%3AAcryl%2520Data%2CselectionType%3AINCLUDED)%2C(text%3ASocket%2CselectionType%3AINCLUDED)%2C(text%3AFelt%2CselectionType%3AINCLUDED)%2C(text%3AXata.io%2CselectionType%3AINCLUDED)%2C(text%3ASuccinct%2CselectionType%3AINCLUDED)%2C(text%3ACognition%2CselectionType%3AINCLUDED)%2C(text%3ANile%2CselectionType%3AINCLUDED)%2C(text%3ARecraft%2CselectionType%3AINCLUDED)%2C(text%3AOrchid%2CselectionType%3AINCLUDED)%2C(text%3AStarkWare%2CselectionType%3AINCLUDED)%2C(text%3AOpendoor%2CselectionType%3AINCLUDED)%2C(text%3AOneSchema%2CselectionType%3AINCLUDED)%2C(text%3AFarcaster%2CselectionType%3AINCLUDED)%2C(text%3ACopy.ai%2CselectionType%3AINCLUDED)%2C(text%3ATriNet%2520Zenefits%2CselectionType%3AINCLUDED)%2C(text%3AKlarity%2CselectionType%3AINCLUDED)%2C(text%3AZeta%2520Labs%2CselectionType%3AINCLUDED)%2C(text%3ASuno%2CselectionType%3AINCLUDED)%2C(text%3AEspresso%2520AI%2CselectionType%3AINCLUDED)%2C(text%3AMatic%2520Robots%2CselectionType%3AINCLUDED)%2C(text%3AReka%2520AI%2CselectionType%3AINCLUDED)%2C(text%3ADigits%2CselectionType%3AINCLUDED)%2C(text%3AZepto%2CselectionType%3AINCLUDED)%2C(text%3ARupa%2520Health%2CselectionType%3AINCLUDED)%2C(text%3AConvictional%2CselectionType%3AINCLUDED)%2C(text%3ABRINC%2CselectionType%3AINCLUDED)%2C(text%3AVoiceflow%2CselectionType%3AINCLUDED)%2C(text%3AThe%2520Browser%2520Company%2CselectionType%3AINCLUDED)%2C(text%3AKapwing%2CselectionType%3AINCLUDED)%2C(text%3ACartwheel%2CselectionType%3AINCLUDED)%2C(text%3APocus%2CselectionType%3AINCLUDED)%2C(text%3AKhatabook%2CselectionType%3AINCLUDED)%2C(text%3ALightspin%2520%2528Acquired%2520by%2520Cisco%2529%2CselectionType%3AINCLUDED)))%2C(type%3AINDUSTRY%2Cvalues%3AList((id%3A10%2Ctext%3ALegal%2520Services%2CselectionType%3AEXCLUDED)%2C(id%3A1080%2Ctext%3AHousehold%2520and%2520Institutional%2520Furniture%2520Manufacturing%2CselectionType%3AEXCLUDED)%2C(id%3A1095%2Ctext%3AMattress%2520and%2520Blinds%2520Manufacturing%2CselectionType%3AEXCLUDED)%2C(id%3A2054%2Ctext%3APhysical%252C%2520Occupational%2520and%2520Speech%2520Therapists%2CselectionType%3AEXCLUDED)%2C(id%3A3126%2Ctext%3AInterior%2520Design%2CselectionType%3AEXCLUDED)%2C(id%3A3132%2Ctext%3AInternet%2520Publishing%2CselectionType%3AEXCLUDED)%2C(id%3A57%2Ctext%3AOil%2520and%2520Gas%2CselectionType%3AEXCLUDED)%2C(id%3A61%2Ctext%3APaper%2520and%2520Forest%2520Product%2520Manufacturing%2CselectionType%3AEXCLUDED)%2C(id%3A62%2Ctext%3ARailroad%2520Equipment%2520Manufacturing%2CselectionType%3AEXCLUDED)%2C(id%3A64%2Ctext%3ARanching%2CselectionType%3AEXCLUDED)%2C(id%3A66%2Ctext%3AFisheries%2CselectionType%3AEXCLUDED)%2C(id%3A7%2Ctext%3ASemiconductor%2520Manufacturing%2CselectionType%3AEXCLUDED)%2C(id%3A71%2Ctext%3AArmed%2520Forces%2CselectionType%3AEXCLUDED)%2C(id%3A72%2Ctext%3ALegislative%2520Offices%2CselectionType%3AEXCLUDED)%2C(id%3A8%2Ctext%3ATelecommunications%2CselectionType%3AEXCLUDED)))%2C(type%3AFUNCTION%2Cvalues%3AList((id%3A1%2Ctext%3AAccounting%2CselectionType%3AEXCLUDED)%2C(id%3A12%2Ctext%3AHuman%2520Resources%2CselectionType%3AEXCLUDED)%2C(id%3A15%2Ctext%3AMarketing%2CselectionType%3AEXCLUDED)%2C(id%3A2%2Ctext%3AAdministrative%2CselectionType%3AEXCLUDED)%2C(id%3A26%2Ctext%3ACustomer%2520Success%2520and%2520Support%2CselectionType%3AEXCLUDED)%2C(id%3A3%2Ctext%3AArts%2520and%2520Design%2CselectionType%3AEXCLUDED)))%2C(type%3ACOMPANY_HEADQUARTERS%2Cvalues%3AList((id%3A103644278%2Ctext%3AUnited%2520States%2CselectionType%3AINCLUDED)%2C(id%3A100506914%2Ctext%3AEurope%2CselectionType%3AINCLUDED)%2C(id%3A101174742%2Ctext%3ACanada%2CselectionType%3AINCLUDED)%2C(id%3A101620260%2Ctext%3AIsrael%2CselectionType%3AINCLUDED)%2C(id%3A102713980%2Ctext%3AIndia%2CselectionType%3AEXCLUDED)%2C(id%3A103537801%2Ctext%3AAfrica%2CselectionType%3AEXCLUDED)%2C(id%3A104305776%2Ctext%3AUnited%2520Arab%2520Emirates%2CselectionType%3AEXCLUDED)))%2C(type%3AYEARS_IN_CURRENT_POSITION%2Cvalues%3AList((id%3A1%2Ctext%3ALess%2520than%25201%2520year%2CselectionType%3AINCLUDED)))%2C(type%3APROFILE_LANGUAGE%2Cvalues%3AList((id%3Aen%2Ctext%3AEnglish%2CselectionType%3AINCLUDED)))%2C(type%3ALEAD_INTERACTIONS%2Cvalues%3AList((id%3ALIMP%2Ctext%3AMessaged%2CselectionType%3AEXCLUDED)))%2C(type%3ACURRENT_TITLE%2Cvalues%3AList((id%3A35%2Ctext%3AFounder%2CselectionType%3AINCLUDED)%2C(id%3A103%2Ctext%3ACo-Founder%2CselectionType%3AINCLUDED)%2C(id%3A8%2Ctext%3AChief%2520Executive%2520Officer%2CselectionType%3AINCLUDED)%2C(id%3A153%2Ctext%3AChief%2520Technology%2520Officer%2CselectionType%3AINCLUDED))))%2Ckeywords%3A%2522something%2520new%2522%2520OR%2520%2522stealth%2522)&sessionId=zVKTgf2GQk2owfFVzL9aFQ%3D%3D
"""

### Part 1: Sales Nav Scraper

In [ ]:
# Sales Nav Scraping
driver = instantiate_driver()
already_scraped_urls, to_scrape_urls = check_and_load_pickles(historyPickle, toScrapePickle)
to_scrape_urls = scrape_profiles(driver, SALES_NAV_SEARCH_URL, already_scraped_urls, to_scrape_urls, toScrapePickle)
print(str(len(to_scrape_urls)) + "profiles identified for scraping") 

### Part 2: Profile scraper, export data to Google Sheets, and reset results pickle

In [ ]:
# Profile Scraping
deduplicate(toScrapePickle, to_scrape_urls, already_scraped_urls)
results = scrape_all_profiles(driver, historyPickle, toScrapePickle, resultsPickle, failedURLsTextFile)


# Export results to Google Sheets
worksheet_title = 'Portfolio'
resultsPickle = 'db/portfolio_results.pickle'
results = readPickle(resultsPickle)
if results:
    worksheet_url = export_sheet(results, spreadsheet, worksheet_title)
    print(f"Worksheet updated: {worksheet_url}")
else:
    print("No data to export.")


# Reset results
results = []
writePickle(resultsPickle, results)
driver.close()

# QUERY 4: UNICORN COMPANIES

### Setup

In [ ]:
toScrapePickle = 'db/unicorn_to_scrape.pickle'
resultsPickle = 'db/unicorn_results.pickle'
failedURLsTextFile = 'db/unicorn_failed_urls.txt'
historyPickle = 'db/history.pickle'

SALES_NAV_SEARCH_URL = """
https://www.linkedin.com/sales/search/people#query=(spellCorrectionEnabled%3Atrue%2CrecentSearchParam%3A(id%3A3934788436%2CdoLogHistory%3Atrue)%2Cfilters%3AList((type%3ACURRENT_COMPANY%2Cvalues%3AList((id%3Aurn%253Ali%253Aorganization%253A91313799%2Ctext%3AStealth%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A18583501%2Ctext%3AStealth%2520Startup%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A18016269%2Ctext%3AStealth%2520Mode%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A96670793%2Ctext%3AStealth%2520AI%2520Startup%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A31285650%2Ctext%3ANewCo%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))))%2C(type%3APAST_COMPANY%2Cvalues%3AList((text%3ACedar%2CselectionType%3AINCLUDED)%2C(text%3AUal%25C3%25A1%2CselectionType%3AINCLUDED)%2C(text%3AAirwallex%2CselectionType%3AINCLUDED)%2C(text%3AImmutable%2CselectionType%3AINCLUDED)%2C(text%3ASafetyCulture%2CselectionType%3AINCLUDED)%2C(text%3ACulture%2520Amp%2CselectionType%3AINCLUDED)%2C(text%3ALinktree%2CselectionType%3AINCLUDED)%2C(text%3ACollibra%2CselectionType%3AINCLUDED)%2C(text%3AQuintoAndar%2CselectionType%3AINCLUDED)%2C(text%3ACreditas%2CselectionType%3AINCLUDED)%2C(text%3ACloudWalk%252C%2520Inc.%2CselectionType%3AINCLUDED)%2C(text%3ALoft%2CselectionType%3AINCLUDED)%2C(text%3ACargoX%2CselectionType%3AINCLUDED)%2C(text%3AMadeiraMadeira%2CselectionType%3AINCLUDED)%2C(text%3ADapper%2520Labs%2CselectionType%3AINCLUDED)%2C(text%3ARappi%2CselectionType%3AINCLUDED)%2C(text%3AHabi%2CselectionType%3AINCLUDED)%2C(text%3APleo%2CselectionType%3AINCLUDED)%2C(text%3AKushki%2CselectionType%3AINCLUDED)%2C(text%3ABolt%2CselectionType%3AINCLUDED)%2C(text%3AVeriff%2CselectionType%3AINCLUDED)%2C(text%3AAiven%2CselectionType%3AINCLUDED)%2C(text%3A%25C5%258CURA%2CselectionType%3AINCLUDED)%2C(text%3AContentsquare%2CselectionType%3AINCLUDED)%2C(text%3AQonto%2CselectionType%3AINCLUDED)%2C(text%3ASorare%2CselectionType%3AINCLUDED)%2C(text%3ABlaBlaCar%2CselectionType%3AINCLUDED)%2C(text%3ASpendesk%2CselectionType%3AINCLUDED)%2C(text%3ALedger%2CselectionType%3AINCLUDED)%2C(text%3ASwile%2CselectionType%3AINCLUDED)%2C(text%3AShift%2520Technology%2CselectionType%3AINCLUDED)%2C(text%3ACelonis%2CselectionType%3AINCLUDED)%2C(text%3AN26%2CselectionType%3AINCLUDED)%2C(text%3APersonio%2CselectionType%3AINCLUDED)%2C(text%3ATrade%2520Republic%2CselectionType%3AINCLUDED)%2C(text%3Awefox%2CselectionType%3AINCLUDED)%2C(text%3AContentful%2CselectionType%3AINCLUDED)%2C(text%3AHelsing%2CselectionType%3AINCLUDED)%2C(text%3AStaffbase%2CselectionType%3AINCLUDED)%2C(text%3AFlink%2CselectionType%3AINCLUDED)%2C(text%3ADeepL%2CselectionType%3AINCLUDED)%2C(text%3ATaxfix%2CselectionType%3AINCLUDED)%2C(text%3AAgile%2520Robots%2520SE%2CselectionType%3AINCLUDED)%2C(text%3ABrowserStack%2CselectionType%3AINCLUDED)%2C(text%3ACato%2520Networks%2CselectionType%3AINCLUDED)%2C(text%3ATransmit%2520Security%2CselectionType%3AINCLUDED)%2C(text%3ADriveNets%2CselectionType%3AINCLUDED)%2C(text%3AHiBob%2CselectionType%3AINCLUDED)%2C(text%3AFirebolt%2CselectionType%3AINCLUDED)%2C(text%3AAqua%2520Security%2CselectionType%3AINCLUDED)%2C(text%3ANord%2520Security%2CselectionType%3AINCLUDED)%2C(text%3AKavak.com%2CselectionType%3AINCLUDED)%2C(text%3ABitso%2CselectionType%3AINCLUDED)%2C(text%3AKonf%25C3%25ADo%2CselectionType%3AINCLUDED)%2C(text%3ANowports%2CselectionType%3AINCLUDED)%2C(text%3AClara%2CselectionType%3AINCLUDED)%2C(text%3AMambu%2CselectionType%3AINCLUDED)%2C(text%3ASparkPost%252C%2520a%2520MessageBird%2520company%2CselectionType%3AINCLUDED)%2C(text%3ADune%2CselectionType%3AINCLUDED)%2C(text%3ARevolut%2CselectionType%3AINCLUDED)%2C(text%3ACheckout.com%2CselectionType%3AINCLUDED)%2C(text%3ARapyd%2CselectionType%3AINCLUDED)%2C(text%3AGoCardless%2CselectionType%3AINCLUDED)%2C(text%3APayFit%2520UK%2CselectionType%3AINCLUDED)%2C(text%3AMultiverse%2CselectionType%3AINCLUDED)%2C(text%3AMatillion%2CselectionType%3AINCLUDED)%2C(text%3ASynthesia%2CselectionType%3AINCLUDED)%2C(text%3AStability%2520AI%2CselectionType%3AINCLUDED)%2C(text%3AEpic%2520Games%2CselectionType%3AINCLUDED)%2C(text%3AGopuff%2CselectionType%3AINCLUDED)%2C(text%3ARipple%2CselectionType%3AINCLUDED)%2C(text%3ADiscord%2CselectionType%3AINCLUDED)%2C(text%3AOpenSea%2CselectionType%3AINCLUDED)%2C(text%3AGrammarly%2CselectionType%3AINCLUDED)%2C(text%3ADevoted%2520Health%2CselectionType%3AINCLUDED)%2C(text%3ADigital%2520Currency%2520Group%2CselectionType%3AINCLUDED)%2C(text%3ANiantic%252C%2520Inc.%2CselectionType%3AINCLUDED)%2C(text%3ATanium%2CselectionType%3AINCLUDED)%2C(text%3ANuro%2CselectionType%3AINCLUDED)%2C(text%3ATipalti%2CselectionType%3AINCLUDED)%2C(text%3AFalconX%2CselectionType%3AINCLUDED)%2C(text%3ANetskope%2CselectionType%3AINCLUDED)%2C(text%3AAutomattic%2CselectionType%3AINCLUDED)%2C(text%3AGemini%2CselectionType%3AINCLUDED)%2C(text%3AConsensys%2CselectionType%3AINCLUDED)%2C(text%3ARo%2CselectionType%3AINCLUDED)%2C(text%3ADataRobot%2CselectionType%3AINCLUDED)%2C(text%3AHinge%2520Health%2CselectionType%3AINCLUDED)%2C(text%3ACommure%2CselectionType%3AINCLUDED)%2C(text%3AApplied%2520Intuition%2CselectionType%3AINCLUDED)%2C(text%3AAttentive%2CselectionType%3AINCLUDED)%2C(text%3ALyra%2520Health%2CselectionType%3AINCLUDED)%2C(text%3AThe%2520Boring%2520Company%2CselectionType%3AINCLUDED)%2C(text%3ACityblock%2520Health%2CselectionType%3AINCLUDED)%2C(text%3ARedwood%2520Materials%2CselectionType%3AINCLUDED)%2C(text%3ASambaNova%2520Systems%2CselectionType%3AINCLUDED)%2C(text%3ACerebral%2CselectionType%3AINCLUDED)%2C(text%3ABetterUp%2CselectionType%3AINCLUDED)%2C(text%3AColor%2CselectionType%3AINCLUDED)%2C(text%3ACheckr%252C%2520Inc.%2CselectionType%3AINCLUDED)%2C(text%3ASocure%2CselectionType%3AINCLUDED)%2C(text%3AOneTrust%2CselectionType%3AINCLUDED)%2C(text%3AEntangled%2520Group%2520%2528now%2520Guild%2520Education%2529%2CselectionType%3AINCLUDED)%2C(text%3Adbt%2520Labs%2CselectionType%3AINCLUDED)%2C(text%3AThoughtSpot%2CselectionType%3AINCLUDED)%2C(text%3AInflection%2520AI%2CselectionType%3AINCLUDED)%2C(text%3AAurora%2520Solar%2CselectionType%3AINCLUDED)%2C(text%3AMelio%2CselectionType%3AINCLUDED)%2C(text%3AClickUp%2CselectionType%3AINCLUDED)%2C(text%3ANext%2520Insurance%2CselectionType%3AINCLUDED)%2C(text%3ACerebras%2520Systems%2CselectionType%3AINCLUDED)%2C(text%3ABranch%2CselectionType%3AINCLUDED)%2C(text%3ARubrik%2CselectionType%3AINCLUDED)%2C(text%3AHouzz%2CselectionType%3AINCLUDED)%2C(text%3AWhatnot%2CselectionType%3AINCLUDED)%2C(text%3APapaya%2520Global%2CselectionType%3AINCLUDED)%2C(text%3AHarness%2CselectionType%3AINCLUDED)%2C(text%3ADataiku%2CselectionType%3AINCLUDED)%2C(text%3ACohesity%2CselectionType%3AINCLUDED)%2C(text%3AFlock%2520Safety%2CselectionType%3AINCLUDED)%2C(text%3ARec%2520Room%2CselectionType%3AINCLUDED)%2C(text%3AMoonPay%2CselectionType%3AINCLUDED)%2C(text%3AKomodo%2520Health%2CselectionType%3AINCLUDED)%2C(text%3ABlockdaemon%2CselectionType%3AINCLUDED)%2C(text%3AThumbtack%2CselectionType%3AINCLUDED)%2C(text%3ALayerZero%2520Labs%2CselectionType%3AINCLUDED)%2C(text%3AAnchorage%2520Digital%2CselectionType%3AINCLUDED)%2C(text%3ACarbon%2520Health%2CselectionType%3AINCLUDED)%2C(text%3ARemote%2CselectionType%3AINCLUDED)%2C(text%3ABlockFi%2CselectionType%3AINCLUDED)%2C(text%3AForter%2CselectionType%3AINCLUDED)%2C(text%3APodium%2CselectionType%3AINCLUDED)%2C(text%3AKraken%2CselectionType%3AINCLUDED)%2C(text%3AWorkrise%2CselectionType%3AINCLUDED)%2C(text%3AMotive%2CselectionType%3AINCLUDED)%2C(text%3AIllumio%2CselectionType%3AINCLUDED)%2C(text%3AEthos%2CselectionType%3AINCLUDED)%2C(text%3Aproject44%2CselectionType%3AINCLUDED)%2C(text%3AFreenome%2CselectionType%3AINCLUDED)%2C(text%3AClari%2CselectionType%3AINCLUDED)%2C(text%3APendo.io%2CselectionType%3AINCLUDED)%2C(text%3ASiFive%2CselectionType%3AINCLUDED)%2C(text%3ASpring%2520Health%2CselectionType%3AINCLUDED)%2C(text%3ASysdig%2CselectionType%3AINCLUDED)%2C(text%3Ainsitro%2CselectionType%3AINCLUDED)%2C(text%3AExabeam%2CselectionType%3AINCLUDED)%2C(text%3AABL%2520Space%2520Systems%2CselectionType%3AINCLUDED)%2C(text%3ACarbon%2CselectionType%3AINCLUDED)%2C(text%3ASkydio%2CselectionType%3AINCLUDED)%2C(text%3ADialpad%2CselectionType%3AINCLUDED)%2C(text%3AAddepar%2CselectionType%3AINCLUDED)%2C(text%3AQualia%2CselectionType%3AINCLUDED)%2C(text%3AJeeves%2CselectionType%3AINCLUDED)%2C(text%3AMoveworks%2CselectionType%3AINCLUDED)%2C(text%3AMysten%2520Labs%2CselectionType%3AINCLUDED)%2C(text%3ACertiK%2CselectionType%3AINCLUDED)%2C(text%3ASword%2520Health%2CselectionType%3AINCLUDED)%2C(text%3AAviatrix%2CselectionType%3AINCLUDED)%2C(text%3ADivvy%2520Homes%2CselectionType%3AINCLUDED)%2C(text%3AMural%2CselectionType%3AINCLUDED)%2C(text%3ANeo4j%2CselectionType%3AINCLUDED)%2C(text%3AIterable%2CselectionType%3AINCLUDED)%2C(text%3AVirta%2520Health%2CselectionType%3AINCLUDED)%2C(text%3AInstabase%2CselectionType%3AINCLUDED)%2C(text%3AMagic%2520Leap%2CselectionType%3AINCLUDED)%2C(text%3AInVision%2CselectionType%3AINCLUDED)%2C(text%3AZocdoc%2CselectionType%3AINCLUDED)%2C(text%3ABitGo%2CselectionType%3AINCLUDED)%2C(text%3ACrusoe%2CselectionType%3AINCLUDED)%2C(text%3ARedesign%2520Health%2CselectionType%3AINCLUDED)%2C(text%3AZenBusiness%2CselectionType%3AINCLUDED)%2C(text%3AH2O.ai%2CselectionType%3AINCLUDED)%2C(text%3AAlation%2CselectionType%3AINCLUDED)%2C(text%3ACircleCI%2CselectionType%3AINCLUDED)%2C(text%3AIncredible%2520Health%2CselectionType%3AINCLUDED)%2C(text%3AApollo.io%2CselectionType%3AINCLUDED)%2C(text%3APave%2CselectionType%3AINCLUDED)%2C(text%3AMagic%2520Eden%2CselectionType%3AINCLUDED)%2C(text%3ACresta%2CselectionType%3AINCLUDED)%2C(text%3ATruepill%2CselectionType%3AINCLUDED)%2C(text%3ADataStax%2CselectionType%3AINCLUDED)%2C(text%3AAstranis%2520Space%2520Technologies%2CselectionType%3AINCLUDED)%2C(text%3AAlloy%2CselectionType%3AINCLUDED)%2C(text%3ARunway%2CselectionType%3AINCLUDED)%2C(text%3AAirbyte%2CselectionType%3AINCLUDED)%2C(text%3ACoinList%2CselectionType%3AINCLUDED)%2C(text%3ATaxBit%2CselectionType%3AINCLUDED)%2C(text%3ASnapdocs%2CselectionType%3AINCLUDED)%2C(text%3AK%2520Health%2CselectionType%3AINCLUDED)%2C(text%3AStrava%2CselectionType%3AINCLUDED)%2C(text%3ACollective%2520Health%2CselectionType%3AINCLUDED)%2C(text%3AAleo%2CselectionType%3AINCLUDED)%2C(text%3AM1%2CselectionType%3AINCLUDED)%2C(text%3AJustworks%2CselectionType%3AINCLUDED)%2C(text%3ANimbleRx%2CselectionType%3AINCLUDED)%2C(text%3AClarify%2520Health%2520Solutions%2CselectionType%3AINCLUDED)%2C(text%3ASalt%2520Security%2CselectionType%3AINCLUDED)%2C(text%3AEnvoy%2CselectionType%3AINCLUDED)%2C(text%3AVideoAmp%2CselectionType%3AINCLUDED)%2C(text%3ACoda%2CselectionType%3AINCLUDED)%2C(text%3AStash%2CselectionType%3AINCLUDED)%2C(text%3AKong%2520Inc.%2CselectionType%3AINCLUDED)%2C(text%3APatreon%2CselectionType%3AINCLUDED)%2C(text%3AZ%25C5%25ABm%2CselectionType%3AINCLUDED)%2C(text%3AClipboard%2520Health%2CselectionType%3AINCLUDED)%2C(text%3ACoinTracker%2CselectionType%3AINCLUDED)%2C(text%3AYugabyte%2CselectionType%3AINCLUDED)%2C(text%3ABetterment%2CselectionType%3AINCLUDED)%2C(text%3AStord%2CselectionType%3AINCLUDED)%2C(text%3AEverly%2520Health%2CselectionType%3AINCLUDED)%2C(text%3AIntercom%2CselectionType%3AINCLUDED)%2C(text%3APolyhedra%2520Network%2CselectionType%3AINCLUDED)%2C(text%3AClassDojo%2CselectionType%3AINCLUDED)%2C(text%3ATalos%2CselectionType%3AINCLUDED)%2C(text%3ACaptivateIQ%2CselectionType%3AINCLUDED)%2C(text%3ATackle.io%2CselectionType%3AINCLUDED)%2C(text%3AEscan%2520Technologies%2520%252F%2520Incode%2520Corp%2520%252F%2520Exeba%2CselectionType%3AINCLUDED)%2C(text%3ABigID%2CselectionType%3AINCLUDED)%2C(text%3ASeatGeek%2CselectionType%3AINCLUDED)%2C(text%3AKallyope%2CselectionType%3AINCLUDED)%2C(text%3ALightmatter%2CselectionType%3AINCLUDED)%2C(text%3AUnit%2CselectionType%3AINCLUDED)%2C(text%3AViz.ai%2CselectionType%3AINCLUDED)%2C(text%3APhantom%2CselectionType%3AINCLUDED)%2C(text%3ABigPanda%2CselectionType%3AINCLUDED)%2C(text%3AAgentSync%2CselectionType%3AINCLUDED)%2C(text%3AGem%2CselectionType%3AINCLUDED)%2C(text%3AOffchain%2520Labs%2CselectionType%3AINCLUDED)%2C(text%3AFloQast%2CselectionType%3AINCLUDED)%2C(text%3AVectra%2520AI%2CselectionType%3AINCLUDED)%2C(text%3APilot%2CselectionType%3AINCLUDED)%2C(text%3AColossal%2520Biosciences%2CselectionType%3AINCLUDED)%2C(text%3AModern%2520Health%2CselectionType%3AINCLUDED)%2C(text%3ACloudBees%2CselectionType%3AINCLUDED)%2C(text%3AELEMY%2CselectionType%3AINCLUDED)%2C(text%3AGuideline%2CselectionType%3AINCLUDED)%2C(text%3AVenafi%2CselectionType%3AINCLUDED)%2C(text%3AEnable%2CselectionType%3AINCLUDED)%2C(text%3AAt-Bay%2CselectionType%3AINCLUDED)%2C(text%3AImply%2CselectionType%3AINCLUDED)%2C(text%3ATeleport%2CselectionType%3AINCLUDED)%2C(text%3ATuring%2CselectionType%3AINCLUDED)%2C(text%3AFundbox%2CselectionType%3AINCLUDED)%2C(text%3AKarat%2CselectionType%3AINCLUDED)%2C(text%3ASisense%2CselectionType%3AINCLUDED)%2C(text%3A0x%2CselectionType%3AINCLUDED)%2C(text%3AMixpanel%2CselectionType%3AINCLUDED)%2C(text%3ADensity%2CselectionType%3AINCLUDED)%2C(text%3ASendbird%2CselectionType%3AINCLUDED)%2C(text%3AVestwell%2CselectionType%3AINCLUDED)%2C(text%3AHeadway%2CselectionType%3AINCLUDED)%2C(text%3AImbue%2CselectionType%3AINCLUDED)%2C(text%3ATypeface%2CselectionType%3AINCLUDED)%2C(text%3AKin%2520Insurance%2CselectionType%3AINCLUDED)%2C(text%3AKoBold%2520Metals%2CselectionType%3AINCLUDED)%2C(text%3AAdept%2CselectionType%3AINCLUDED)%2C(text%3AFlow%2CselectionType%3AINCLUDED)%2C(text%3ASingleStore%2CselectionType%3AINCLUDED)%2C(text%3AFourKites%252C%2520Inc.%2CselectionType%3AINCLUDED)%2C(text%3AImmuta%2CselectionType%3AINCLUDED)%2C(text%3AJupiterOne%2CselectionType%3AINCLUDED)%2C(text%3AFlexe%2CselectionType%3AINCLUDED)%2C(text%3ANexHealth%2CselectionType%3AINCLUDED)%2C(text%3AOyster%2CselectionType%3AINCLUDED)%2C(text%3AElectric%2CselectionType%3AINCLUDED)%2C(text%3ARapid%2CselectionType%3AINCLUDED)%2C(text%3AGauntlet%2CselectionType%3AINCLUDED)%2C(text%3APalmetto%2CselectionType%3AINCLUDED)%2C(text%3AVendr%2CselectionType%3AINCLUDED)%2C(text%3AOmada%2520Health%2CselectionType%3AINCLUDED)%2C(text%3ATimescale%2CselectionType%3AINCLUDED)%2C(text%3AWatershed%2CselectionType%3AINCLUDED)%2C(text%3ACadence%2CselectionType%3AINCLUDED)%2C(text%3ASnapLogic%2CselectionType%3AINCLUDED)%2C(text%3AAnyscale%2CselectionType%3AINCLUDED)%2C(text%3AExpel%2CselectionType%3AINCLUDED)%2C(text%3AWrapbook%2CselectionType%3AINCLUDED)%2C(text%3Asolo.io%2CselectionType%3AINCLUDED)%2C(text%3AFormation%2520Bio%2CselectionType%3AINCLUDED)%2C(text%3APandaDoc%2CselectionType%3AINCLUDED)%2C(text%3AMammoth%2520Biosciences%2CselectionType%3AINCLUDED)%2C(text%3APicsart%2CselectionType%3AINCLUDED)%2C(text%3AMaven%2520Clinic%2CselectionType%3AINCLUDED)%2C(text%3AHuman%2520Interest%2CselectionType%3AINCLUDED)%2C(text%3AAmperity%2CselectionType%3AINCLUDED)%2C(text%3APantheon%2520Systems%2CselectionType%3AINCLUDED)%2C(text%3AWorldcoin%2CselectionType%3AINCLUDED)%2C(text%3AShipBob%2CselectionType%3AINCLUDED)%2C(text%3AShippo%2CselectionType%3AINCLUDED)%2C(text%3AMux%2CselectionType%3AINCLUDED)%2C(text%3ACapsule%2CselectionType%3AINCLUDED)%2C(text%3ASift%2CselectionType%3AINCLUDED)%2C(text%3AGroq%2CselectionType%3AINCLUDED)%2C(text%3AINJECTIVE%2CselectionType%3AINCLUDED)%2C(text%3ALocus%2520Robotics%2CselectionType%3AINCLUDED)%2C(text%3AAxiom%2520Space%2CselectionType%3AINCLUDED)%2C(text%3ASidecar%2520Health%2CselectionType%3AINCLUDED)%2C(text%3ABoom%2520Supersonic%2CselectionType%3AINCLUDED)%2C(text%3AQuizlet%2CselectionType%3AINCLUDED)%2C(text%3ALOOKOUT%2CselectionType%3AINCLUDED)%2C(text%3ATikTok%2CselectionType%3AINCLUDED)%2C(text%3AAdobe%2CselectionType%3AINCLUDED)%2C(text%3AWix%2CselectionType%3AINCLUDED)%2C(text%3AAsana%2CselectionType%3AINCLUDED)%2C(text%3ABILL%2CselectionType%3AINCLUDED)%2C(text%3AFreshworks%2CselectionType%3AINCLUDED)))%2C(type%3AINDUSTRY%2Cvalues%3AList((id%3A10%2Ctext%3ALegal%2520Services%2CselectionType%3AEXCLUDED)%2C(id%3A1080%2Ctext%3AHousehold%2520and%2520Institutional%2520Furniture%2520Manufacturing%2CselectionType%3AEXCLUDED)%2C(id%3A1095%2Ctext%3AMattress%2520and%2520Blinds%2520Manufacturing%2CselectionType%3AEXCLUDED)%2C(id%3A2054%2Ctext%3APhysical%252C%2520Occupational%2520and%2520Speech%2520Therapists%2CselectionType%3AEXCLUDED)%2C(id%3A3126%2Ctext%3AInterior%2520Design%2CselectionType%3AEXCLUDED)%2C(id%3A3132%2Ctext%3AInternet%2520Publishing%2CselectionType%3AEXCLUDED)%2C(id%3A57%2Ctext%3AOil%2520and%2520Gas%2CselectionType%3AEXCLUDED)%2C(id%3A61%2Ctext%3APaper%2520and%2520Forest%2520Product%2520Manufacturing%2CselectionType%3AEXCLUDED)%2C(id%3A62%2Ctext%3ARailroad%2520Equipment%2520Manufacturing%2CselectionType%3AEXCLUDED)%2C(id%3A64%2Ctext%3ARanching%2CselectionType%3AEXCLUDED)%2C(id%3A66%2Ctext%3AFisheries%2CselectionType%3AEXCLUDED)%2C(id%3A7%2Ctext%3ASemiconductor%2520Manufacturing%2CselectionType%3AEXCLUDED)%2C(id%3A71%2Ctext%3AArmed%2520Forces%2CselectionType%3AEXCLUDED)%2C(id%3A72%2Ctext%3ALegislative%2520Offices%2CselectionType%3AEXCLUDED)%2C(id%3A8%2Ctext%3ATelecommunications%2CselectionType%3AEXCLUDED)))%2C(type%3AREGION%2Cvalues%3AList((id%3A102713980%2Ctext%3AIndia%2CselectionType%3AEXCLUDED)))%2C(type%3AFUNCTION%2Cvalues%3AList((id%3A1%2Ctext%3AAccounting%2CselectionType%3AEXCLUDED)%2C(id%3A12%2Ctext%3AHuman%2520Resources%2CselectionType%3AEXCLUDED)%2C(id%3A15%2Ctext%3AMarketing%2CselectionType%3AEXCLUDED)%2C(id%3A2%2Ctext%3AAdministrative%2CselectionType%3AEXCLUDED)%2C(id%3A26%2Ctext%3ACustomer%2520Success%2520and%2520Support%2CselectionType%3AEXCLUDED)%2C(id%3A3%2Ctext%3AArts%2520and%2520Design%2CselectionType%3AEXCLUDED)))%2C(type%3AYEARS_IN_CURRENT_POSITION%2Cvalues%3AList((id%3A1%2Ctext%3ALess%2520than%25201%2520year%2CselectionType%3AINCLUDED)))%2C(type%3APROFILE_LANGUAGE%2Cvalues%3AList((id%3Aen%2Ctext%3AEnglish%2CselectionType%3AINCLUDED)))%2C(type%3ALEAD_INTERACTIONS%2Cvalues%3AList((id%3ALIMP%2Ctext%3AMessaged%2CselectionType%3AEXCLUDED)%2C(id%3ALIVP%2Ctext%3AViewed%2520profile%2CselectionType%3AEXCLUDED)))%2C(type%3ACURRENT_TITLE%2Cvalues%3AList((id%3A35%2Ctext%3AFounder%2CselectionType%3AINCLUDED)%2C(id%3A103%2Ctext%3ACo-Founder%2CselectionType%3AINCLUDED)%2C(id%3A8%2Ctext%3AChief%2520Executive%2520Officer%2CselectionType%3AINCLUDED)%2C(id%3A153%2Ctext%3AChief%2520Technology%2520Officer%2CselectionType%3AINCLUDED)%2C(id%3A1128%2Ctext%3ABuilder%2CselectionType%3AINCLUDED))))%2Ckeywords%3A%2522something%2520new%2522%2520OR%2520%2522stealth%2522)&sessionId=zVKTgf2GQk2owfFVzL9aFQ%3D%3D
"""

### Part 1: Sales Nav Scraper

In [ ]:
# Sales Nav Scraping
driver = instantiate_driver()
already_scraped_urls, to_scrape_urls = check_and_load_pickles(historyPickle, toScrapePickle)
to_scrape_urls = scrape_profiles(driver, SALES_NAV_SEARCH_URL, already_scraped_urls, to_scrape_urls, toScrapePickle)
print(str(len(to_scrape_urls)) + "profiles identified for scraping") 

### Part 2: Profile scraper, export data to Google Sheets, and reset results pickle

In [ ]:
# Profile Scraping
deduplicate(toScrapePickle, to_scrape_urls, already_scraped_urls)
results = scrape_all_profiles(driver, historyPickle, toScrapePickle, resultsPickle, failedURLsTextFile)


# Export results to Google Sheets
worksheet_title = 'Unicorn'
resultsPickle = 'db/unicorn_results.pickle'
results = readPickle(resultsPickle)
if results:
    worksheet_url = export_sheet(results, spreadsheet, worksheet_title)
    print(f"Worksheet updated: {worksheet_url}")
else:
    print("No data to export.")


# Reset results
results = []
writePickle(resultsPickle, results)
driver.close()

# QUERY 5: ACQUIRED COMPANIES

### Setup

In [ ]:
toScrapePickle = 'db/acquired_to_scrape.pickle'
resultsPickle = 'db/acquired_results.pickle'
failedURLsTextFile = 'db/acquired_failed_urls.txt'
historyPickle = 'db/history.pickle'

SALES_NAV_SEARCH_URL = """
https://www.linkedin.com/sales/search/people#query=(spellCorrectionEnabled%3Atrue%2CrecentSearchParam%3A(id%3A3934788444%2CdoLogHistory%3Atrue)%2Cfilters%3AList((type%3ACURRENT_COMPANY%2Cvalues%3AList((id%3Aurn%253Ali%253Aorganization%253A18583501%2Ctext%3AStealth%2520Startup%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A91313799%2Ctext%3AStealth%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A31285650%2Ctext%3ANewCo%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A91194134%2Ctext%3ASomething%2520new%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))))%2C(type%3APAST_COMPANY%2Cvalues%3AList((text%3ACredit%2520Karma%2CselectionType%3AINCLUDED)%2C(text%3ABionic%252C%2520a%2520CrowdStrike%2520Company%2CselectionType%3AINCLUDED)%2C(text%3ASqreen%2CselectionType%3AINCLUDED)%2C(text%3AFoundationDB%2CselectionType%3AINCLUDED)%2C(text%3ARockset%2CselectionType%3AINCLUDED)%2C(text%3ARedLock%2CselectionType%3AINCLUDED)%2C(text%3AEndgame%2CselectionType%3AINCLUDED)%2C(text%3AAporeto%2CselectionType%3AINCLUDED)%2C(text%3ARundeck%2CselectionType%3AINCLUDED)%2C(text%3ABridgecrew%2CselectionType%3AINCLUDED)%2C(text%3AHumio%2CselectionType%3AINCLUDED)%2C(text%3ASay%2520Technologies%2520%2528acquired%2520by%2520Robinhood%2529%2CselectionType%3AINCLUDED)%2C(text%3AWeav%2CselectionType%3AINCLUDED)%2C(text%3AHVR%2520-%2520a%2520Fivetran%2520Company%2CselectionType%3AINCLUDED)%2C(text%3ACaper%2CselectionType%3AINCLUDED)%2C(text%3ACognito%2CselectionType%3AINCLUDED)%2C(text%3AStreamlit%2CselectionType%3AINCLUDED)%2C(text%3APry%2520Financials%2CselectionType%3AINCLUDED)%2C(text%3ABionic%2CselectionType%3AINCLUDED)%2C(text%3AArcion%2520Labs%2520%2528Acquired%2520by%2520Databricks%2529%2CselectionType%3AINCLUDED)%2C(text%3ADig%2520Security%2520%2528acquired%2520by%2520Palo%2520Alto%2520Networks%2529%2CselectionType%3AINCLUDED)%2C(text%3AClearbit%2CselectionType%3AINCLUDED)%2C(text%3ARun%253Aai%2CselectionType%3AINCLUDED)%2C(text%3ADeci%2520AI%2520%2528Acquired%2520by%2520NVIDIA%2529%2CselectionType%3AINCLUDED)%2C(text%3AReaction%2520Commerce%2520%2528acquired%2520by%2520Mailchimp%2529%2CselectionType%3AINCLUDED)%2C(text%3AQuay.io%2520%2528Acquired%2520CoreOS%2520%252F%2520Red%2520Hat%2529%2CselectionType%3AINCLUDED)%2C(text%3AAccompany%2CselectionType%3AINCLUDED)%2C(text%3ABitTorrent%252C%2520Inc.%2CselectionType%3AINCLUDED)%2C(text%3ASimility%252C%2520A%2520PayPal%2520Service%2CselectionType%3AINCLUDED)%2C(text%3ASpringCM%252C%2520a%2520DocuSign%2520company%2CselectionType%3AINCLUDED)%2C(text%3ACognizant%2520Softvision%2CselectionType%3AINCLUDED)%2C(text%3AHeptio%2CselectionType%3AINCLUDED)%2C(text%3APlanGrid%252C%2520an%2520Autodesk%2520company%2CselectionType%3AINCLUDED)%2C(text%3AHelloSign%2CselectionType%3AINCLUDED)%2C(text%3AAnchor%2CselectionType%3AINCLUDED)%2C(text%3ACaviar%252C%2520Inc.%2CselectionType%3AINCLUDED)%2C(text%3AOnica%2520by%2520Rackspace%2520Technology%2CselectionType%3AINCLUDED)%2C(text%3AXnor.ai%2CselectionType%3AINCLUDED)%2C(text%3ASeal%2520Software%252C%2520a%2520DocuSign%2520Company%2CselectionType%3AINCLUDED)%2C(text%3ACisco%2520ThousandEyes%2CselectionType%3AINCLUDED)%2C(text%3ASignal%2520Sciences%2CselectionType%3AINCLUDED)%2C(text%3APortworx%2520by%2520Pure%2520Storage%2CselectionType%3AINCLUDED)%2C(text%3APayBright%2CselectionType%3AINCLUDED)%2C(text%3AStackRox%2520%2528Acquired%2520by%2520Red%2520Hat%2529%2CselectionType%3AINCLUDED)%2C(text%3ATitle365%2CselectionType%3AINCLUDED)%2C(text%3ATock%2CselectionType%3AINCLUDED)%2C(text%3AReturnly%2CselectionType%3AINCLUDED)%2C(text%3ASparkPost%252C%2520a%2520MessageBird%2520company%2CselectionType%3AINCLUDED)%2C(text%3AChartboost%2CselectionType%3AINCLUDED)%2C(text%3AZipwhip%2CselectionType%3AINCLUDED)%2C(text%3ATwistle%2520by%2520Health%2520Catalyst%2CselectionType%3AINCLUDED)%2C(text%3AFetch%2520Robotics%2520%2528now%2520part%2520of%2520Zebra%2520Technologies%2529%2CselectionType%3AINCLUDED)%2C(text%3AHero%2CselectionType%3AINCLUDED)%2C(text%3AIntSights%252C%2520a%2520Rapid7%2520company%2CselectionType%3AINCLUDED)%2C(text%3AInvoice2go%2CselectionType%3AINCLUDED)%2C(text%3AStocard%2520-%2520A%2520Klarna%2520Group%2520Company%2CselectionType%3AINCLUDED)%2C(text%3ACurrencycloud%2CselectionType%3AINCLUDED)%2C(text%3AParsec%2CselectionType%3AINCLUDED)%2C(text%3AExOne%2CselectionType%3AINCLUDED)%2C(text%3AEpsagon%2CselectionType%3AINCLUDED)%2C(text%3AConvey%2CselectionType%3AINCLUDED)%2C(text%3ALevelset%252C%2520a%2520Procore%2520Company%2CselectionType%3AINCLUDED)%2C(text%3AFivestars%2520by%2520SumUp%2CselectionType%3AINCLUDED)%2C(text%3AAdvanced%2520Circuits%2CselectionType%3AINCLUDED)%2C(text%3ALemonaid%2520Health%2520%2528part%2520of%252023andMe%2529%2CselectionType%3AINCLUDED)%2C(text%3AMetromile%2CselectionType%3AINCLUDED)%2C(text%3AArtemis%2520by%2520Nomi%2520Health%2CselectionType%3AINCLUDED)%2C(text%3ATrifacta%2CselectionType%3AINCLUDED)%2C(text%3ALinode%2CselectionType%3AINCLUDED)%2C(text%3AFirst%2520Digital%2520%2528Acquired%2520by%2520Fireblocks%2529%2CselectionType%3AINCLUDED)%2C(text%3AUserZoom%2520%2528now%2520part%2520of%2520UserTesting%2529%2CselectionType%3AINCLUDED)%2C(text%3AZymergen%2CselectionType%3AINCLUDED)%2C(text%3AImmerok%2CselectionType%3AINCLUDED)%2C(text%3ACloudify.co%2CselectionType%3AINCLUDED)%2C(text%3ACasetext%252C%2520Part%2520of%2520Thomson%2520Reuters%2CselectionType%3AINCLUDED)%2C(text%3AMode%2CselectionType%3AINCLUDED)%2C(text%3APaperspace%2CselectionType%3AINCLUDED)%2C(text%3AGamePlanner.AI%2CselectionType%3AINCLUDED)%2C(text%3ASpiff%2520%2528Acquired%2520by%2520Salesforce%2529%2CselectionType%3AINCLUDED)%2C(text%3AScienceIO%2CselectionType%3AINCLUDED)%2C(text%3ABeeper%2CselectionType%3AINCLUDED)%2C(text%3AEntitle%2520-%2520a%2520BeyondTrust%2520company%2CselectionType%3AINCLUDED)%2C(text%3ASilk%2520Security%252C%2520now%2520part%2520of%2520Armis%2CselectionType%3AINCLUDED)%2C(text%3ALexion%2CselectionType%3AINCLUDED)%2C(text%3AFlatiron%2520Health%2CselectionType%3AINCLUDED)%2C(text%3AMuleSoft%2CselectionType%3AINCLUDED)%2C(text%3AAdobe%2520Marketo%2CselectionType%3AINCLUDED)%2C(text%3ARed%2520Hat%2CselectionType%3AINCLUDED)%2C(text%3AQualtrics%2CselectionType%3AINCLUDED)%2C(text%3ACylance%2520Inc.%2CselectionType%3AINCLUDED)%2C(text%3ADelivery%2520Hero%2CselectionType%3AINCLUDED)%2C(text%3ALooker%2CselectionType%3AINCLUDED)%2C(text%3ATableau%2CselectionType%3AINCLUDED)%2C(text%3AGainsight%2CselectionType%3AINCLUDED)%2C(text%3ADivvy%2520from%2520BILL%2CselectionType%3AINCLUDED)%2C(text%3ACloudera%2CselectionType%3AINCLUDED)%2C(text%3ATink%2CselectionType%3AINCLUDED)%2C(text%3AFrame.io%2CselectionType%3AINCLUDED)%2C(text%3AIntuit%2520Mailchimp%2CselectionType%3AINCLUDED)%2C(text%3AWolt%2CselectionType%3AINCLUDED)%2C(text%3AVeracode%2CselectionType%3AINCLUDED)%2C(text%3AZendesk%2CselectionType%3AINCLUDED)%2C(text%3APing%2520Identity%2CselectionType%3AINCLUDED)%2C(text%3ACoupa%2520Software%2CselectionType%3AINCLUDED)%2C(text%3ASumo%2520Logic%2CselectionType%3AINCLUDED)%2C(text%3APismo%2CselectionType%3AINCLUDED)%2C(text%3ASplunk%2CselectionType%3AINCLUDED)%2C(text%3AOpenGov%2520Inc.%2CselectionType%3AINCLUDED)%2C(text%3AMatterport%2CselectionType%3AINCLUDED)%2C(text%3AHashiCorp%2CselectionType%3AINCLUDED)%2C(text%3ADarktrace%2CselectionType%3AINCLUDED)%2C(text%3ASquarespace%2CselectionType%3AINCLUDED)%2C(text%3AEarn.com%2CselectionType%3AINCLUDED)%2C(text%3ABlue%2520Vision%2520Labs%2CselectionType%3AINCLUDED)%2C(text%3ASignalFx%2520%2528acquired%2520by%2520Splunk%2529%2CselectionType%3AINCLUDED)%2C(text%3ALoom.ai%2520%2528Acquired%2520by%2520Roblox%2529%2CselectionType%3AINCLUDED)%2C(text%3AFat%2520Llama%2CselectionType%3AINCLUDED)%2C(text%3AGorillas.dev%2CselectionType%3AINCLUDED)%2C(text%3AQwak%2CselectionType%3AINCLUDED)%2C(text%3ARockset%2520%2528acquired%2520by%2520OpenAI%2529%2CselectionType%3AINCLUDED)%2C(text%3ABeReal.%2CselectionType%3AINCLUDED)%2C(text%3AThreads%2CselectionType%3AINCLUDED)%2C(text%3AWalkMe%25E2%2584%25A2%2CselectionType%3AINCLUDED)%2C(text%3ATabular%2520%2528now%2520part%2520of%2520Databricks%2529%2CselectionType%3AINCLUDED)%2C(text%3ASplit%2CselectionType%3AINCLUDED)%2C(text%3AUizard%2520by%2520Miro%2520Labs%2CselectionType%3AINCLUDED)%2C(text%3ACardinal%2520Health%2CselectionType%3AINCLUDED)%2C(text%3ADevice42%252C%2520A%2520Freshworks%2520Company%2CselectionType%3AINCLUDED)%2C(text%3Aoak9%2CselectionType%3AINCLUDED)%2C(text%3ACyborg%2520Security%2CselectionType%3AINCLUDED)%2C(text%3ATeampay%2CselectionType%3AINCLUDED)%2C(text%3AMultis%2CselectionType%3AINCLUDED)%2C(text%3AOnfido%2CselectionType%3AINCLUDED)%2C(text%3APartyKit%2520%2528acquired%2520by%2520Cloudflare%2529%2CselectionType%3AINCLUDED)%2C(text%3ABaselime%2520%2528acquired%2520by%2520Cloudflare%2529%2CselectionType%3AINCLUDED)%2C(text%3APopSQL%2CselectionType%3AINCLUDED)%2C(text%3ASpeedb%2CselectionType%3AINCLUDED)%2C(text%3APliant.io%252C%2520an%2520IBM%2520Company%2CselectionType%3AINCLUDED)%2C(text%3AOxeye%2520%2528Part%2520of%2520GitLab%2529%2CselectionType%3AINCLUDED)%2C(text%3AJama%2520Software%2CselectionType%3AINCLUDED)%2C(text%3ADocFox%2520-%2520is%2520now%2520nCino%2CselectionType%3AINCLUDED)%2C(text%3Afinova%2CselectionType%3AINCLUDED)%2C(text%3ADarwinAI%2CselectionType%3AINCLUDED)%2C(text%3ANefeli%2520%2528acquired%2520by%2520Cloudflare%2529%2CselectionType%3AINCLUDED)%2C(text%3AVowel%2520%2528acquired%2520by%2520Zapier%2529%2CselectionType%3AINCLUDED)%2C(text%3AFlow%2520Security%2CselectionType%3AINCLUDED)%2C(text%3APaySpace%2520by%2520Deel%2CselectionType%3AINCLUDED)%2C(text%3ADefinitive%2520Intelligence%2CselectionType%3AINCLUDED)%2C(text%3AKolide%2520by%25201Password%2CselectionType%3AINCLUDED)%2C(text%3AEinblick%2CselectionType%3AINCLUDED)%2C(text%3AVenue%2CselectionType%3AINCLUDED)%2C(text%3AWindsor.io%2CselectionType%3AINCLUDED)%2C(text%3ACalyptia%2CselectionType%3AINCLUDED)%2C(text%3AArmory%2CselectionType%3AINCLUDED)%2C(text%3AElevate%2520Security%2520%2528acquired%2520by%2520Mimecast%2529%2CselectionType%3AINCLUDED)%2C(text%3AIsovalent%2CselectionType%3AINCLUDED)%2C(text%3AExiger%2CselectionType%3AINCLUDED)%2C(text%3AStreamSets%2520Inc.%2CselectionType%3AINCLUDED)%2C(text%3AAtomicJar%252C%2520Inc.%2520%2528acquired%2520by%2520Docker%2529%2CselectionType%3AINCLUDED)%2C(text%3ARoam.ai%2520%2528an%2520Echo%2520company%2529%2CselectionType%3AINCLUDED)%2C(text%3ARaftt%2CselectionType%3AINCLUDED)%2C(text%3AJeli.io%2CselectionType%3AINCLUDED)%2C(text%3ATexts%2CselectionType%3AINCLUDED)%2C(text%3AFlexSim%2520Software%2520Products%252C%2520Inc.%2CselectionType%3AINCLUDED)%2C(text%3AMexPago%2CselectionType%3AINCLUDED)%2C(text%3ARevelstoke%2520Security%2CselectionType%3AINCLUDED)%2C(text%3AHeap%2520%257C%2520by%2520Contentsquare%2CselectionType%3AINCLUDED)%2C(text%3ASpeechly%2CselectionType%3AINCLUDED)%2C(text%3AAPImetrics%2CselectionType%3AINCLUDED)%2C(text%3AGlobal%2520Illumination%252C%2520Inc%2CselectionType%3AINCLUDED)%2C(text%3AImperva%2CselectionType%3AINCLUDED)%2C(text%3ABoost%2520Commerce%2CselectionType%3AINCLUDED)%2C(text%3AStemma%2CselectionType%3AINCLUDED)%2C(text%3AAkita%2520Software%2CselectionType%3AINCLUDED)%2C(text%3AOort%2520%2528now%2520part%2520of%2520Cisco%2529%2CselectionType%3AINCLUDED)%2C(text%3AStackbit%2520%2528acquired%2520by%2520Netlify%2529%2CselectionType%3AINCLUDED)%2C(text%3ABerbix%2520%2528now%2520part%2520of%2520Socure%2529%2CselectionType%3AINCLUDED)%2C(text%3AApptio%252C%2520an%2520IBM%2520Company%2CselectionType%3AINCLUDED)%2C(text%3AArengu%2CselectionType%3AINCLUDED)%2C(text%3Aenso.security%2520%2528Part%2520of%2520Snyk%2529%2CselectionType%3AINCLUDED)%2C(text%3ADiagram%2CselectionType%3AINCLUDED)%2C(text%3ATranspose%2CselectionType%3AINCLUDED)%2C(text%3AEmbark%2520Trucks%2520%2528acquired%2520by%2520Applied%2520Intuition%2529%2CselectionType%3AINCLUDED)%2C(text%3AAirside%252C%2520an%2520Entrust%2520Company%2CselectionType%3AINCLUDED)%2C(text%3ATalend%2CselectionType%3AINCLUDED)%2C(text%3ALaskie%2520%2528acquired%2520by%2520Twitter%2529%2CselectionType%3AINCLUDED)%2C(text%3AOkera%2CselectionType%3AINCLUDED)%2C(text%3Aneosec.com%2CselectionType%3AINCLUDED)%2C(text%3AWorkvivo%2520-%2520by%2520Zoom%2CselectionType%3AINCLUDED)%2C(text%3ASceneBox%2520%2528Acquired%2520by%2520Applied%2520Intuition%2529%2CselectionType%3AINCLUDED)%2C(text%3APyroscope%2CselectionType%3AINCLUDED)%2C(text%3AOndat%2CselectionType%3AINCLUDED)%2C(text%3AOverOps%2CselectionType%3AINCLUDED)%2C(text%3AOmniML%2CselectionType%3AINCLUDED)%2C(text%3AConductor%2520Technologies%2CselectionType%3AINCLUDED)%2C(text%3APropelo%2520acquired%2520by%2520Harness%2CselectionType%3AINCLUDED)%2C(text%3ATactic%2CselectionType%3AINCLUDED)%2C(text%3AKaskada%2520%2528acquired%2520by%2520DataStax%2529%2CselectionType%3AINCLUDED)%2C(text%3AFuzzbuzz%2CselectionType%3AINCLUDED)%2C(text%3AFungible%2520Inc.%252C%2520Acquired%2520by%2520Microsoft%2CselectionType%3AINCLUDED)%2C(text%3AIllusive%2CselectionType%3AINCLUDED)%2C(text%3ACrashtest%2520Security%2520GmbH%2CselectionType%3AINCLUDED)%2C(text%3AOutwrite%2CselectionType%3AINCLUDED)%2C(text%3ASpirit%2520AI%2CselectionType%3AINCLUDED)%2C(text%3APayGroup%2520by%2520Deel%2CselectionType%3AINCLUDED)%2C(text%3AZoona%2CselectionType%3AINCLUDED)%2C(text%3AAnanda%2520Networks%2520-%2520Now%2520part%2520of%2520VMware%2CselectionType%3AINCLUDED)%2C(text%3ANapkin%2CselectionType%3AINCLUDED)%2C(text%3ARemix%2CselectionType%3AINCLUDED)%2C(text%3ARevv%2CselectionType%3AINCLUDED)%2C(text%3ADatajoy%2CselectionType%3AINCLUDED)%2C(text%3AKnowBe4%2CselectionType%3AINCLUDED)%2C(text%3AForgeRock%2CselectionType%3AINCLUDED)%2C(text%3APrelude%2CselectionType%3AINCLUDED)%2C(text%3Arelay%2520platform%2CselectionType%3AINCLUDED)%2C(text%3ASeradata%2CselectionType%3AINCLUDED)%2C(text%3APluribus%2520Networks%2CselectionType%3AINCLUDED)%2C(text%3AVarada%2CselectionType%3AINCLUDED)%2C(text%3ACron%2CselectionType%3AINCLUDED)%2C(text%3ANestybox%2CselectionType%3AINCLUDED)%2C(text%3APullRequest%2CselectionType%3AINCLUDED)%2C(text%3ABit%2520Discovery%2520%2528Acquired%2520by%2520Tenable%2529%2CselectionType%3AINCLUDED)%2C(text%3AWorkbase%2CselectionType%3AINCLUDED)%2C(text%3APresize%2520%2528acq.%2520by%2520Meta%2529%2CselectionType%3AINCLUDED)%2C(text%3AMIBTech%2520FZ%2520LLC%2520-%2520Barracuda%2520Networks%2CselectionType%3AINCLUDED)%2C(text%3ASailPoint%2CselectionType%3AINCLUDED)%2C(text%3APuppet%2CselectionType%3AINCLUDED)%2C(text%3AGoodHire%2520%2528a%2520Checkr%2520company%2529%2CselectionType%3AINCLUDED)%2C(text%3AGrouparoo%2520%2528acquired%2520by%2520Airbyte%2529%2CselectionType%3AINCLUDED)%2C(text%3ANextMind%2CselectionType%3AINCLUDED)%2C(text%3AAttivo%2520Networks%25C2%25AE%252C%2520a%2520SentinelOne%2520Company%2CselectionType%3AINCLUDED)%2C(text%3AVeeqo%2CselectionType%3AINCLUDED)%2C(text%3AEdgecast%2CselectionType%3AINCLUDED)%2C(text%3ACatalytic%2CselectionType%3AINCLUDED)%2C(text%3ANext%2520Games%252C%2520A%2520Netflix%2520Game%2520Studio%2CselectionType%3AINCLUDED)%2C(text%3ATroops%2520%2528Acquired%2520by%2520Salesforce%2520-%2520July%25202022%2529%2CselectionType%3AINCLUDED)%2C(text%3AFugue%2520%2528part%2520of%2520Snyk%2529%2CselectionType%3AINCLUDED)%2C(text%3ACoScreen%2520%2528acq.%2520by%2520Datadog%2529%2CselectionType%3AINCLUDED)%2C(text%3ACitrix%2CselectionType%3AINCLUDED)%2C(text%3APercept.AI%2520%2528Acquired%2520by%2520Atlassian%2529%2CselectionType%3AINCLUDED)%2C(text%3ARapidSec%2520%2528Acq.%2520by%2520Orca%2520Security%2529%2CselectionType%3AINCLUDED)%2C(text%3ABright%2520Computing%2CselectionType%3AINCLUDED)%2C(text%3AStream%2520Club%2CselectionType%3AINCLUDED)%2C(text%3Awalrus.ai%2CselectionType%3AINCLUDED)%2C(text%3AOffice%2520Ally%2CselectionType%3AINCLUDED)%2C(text%3A%25E5%25BC%25A5%25E7%2594%259F%25E6%25A0%25AA%25E5%25BC%258F%25E4%25BC%259A%25E7%25A4%25BE%25EF%25BC%2588Yayoi%2520Co.%252C%2520Ltd.%25EF%25BC%2589%2CselectionType%3AINCLUDED)%2C(text%3ABottomline%2CselectionType%3AINCLUDED)%2C(text%3AOracle%2CselectionType%3AINCLUDED)%2C(text%3AArgon%2520Security%2CselectionType%3AINCLUDED)%2C(text%3AVectorly%2CselectionType%3AINCLUDED)%2C(text%3ASyncSketch%2CselectionType%3AINCLUDED)%2C(text%3AOneGraph%2CselectionType%3AINCLUDED)%2C(text%3ANarrative%2520Science%2CselectionType%3AINCLUDED)%2C(text%3AAmixr%2520Inc.%2CselectionType%3AINCLUDED)%2C(text%3ASoluble%2CselectionType%3AINCLUDED)%2C(text%3AHELIX%2520RE%2CselectionType%3AINCLUDED)%2C(text%3AEarth%2520Class%2520Mail%2CselectionType%3AINCLUDED)%2C(text%3ARemoteteam.com%2520%2528Acquired%2520by%2520Gusto%2529%2CselectionType%3AINCLUDED)%2C(text%3ALore%2520IO%2520%2528Acquired%2520by%2520Alteryx%2529%2CselectionType%3AINCLUDED)%2C(text%3AKoopid%2CselectionType%3AINCLUDED)%2C(text%3AHoss%2520%2528Acquired%2520by%2520Niantic%252C%2520Inc.%2529%2CselectionType%3AINCLUDED)%2C(text%3AHabitu8%2CselectionType%3AINCLUDED)%2C(text%3AGuardicore%2CselectionType%3AINCLUDED)%2C(text%3ASS%2526C%2520Blue%2520Prism%2CselectionType%3AINCLUDED)%2C(text%3AAccurics%2CselectionType%3AINCLUDED)%2C(text%3AatSpoke%2520%2528Acquired%2520by%2520Okta%2529%2CselectionType%3AINCLUDED)%2C(text%3AScopeAI%2CselectionType%3AINCLUDED)%2C(text%3AAvanan%252C%2520a%2520Check%2520Point%2520Company%2CselectionType%3AINCLUDED)%2C(text%3AFrontity%2CselectionType%3AINCLUDED)%2C(text%3ACleverly.ai%2520%2528acq.%2520by%2520Zendesk%2529%2CselectionType%3AINCLUDED)%2C(text%3ABuyer%2CselectionType%3AINCLUDED)%2C(text%3APredata%2CselectionType%3AINCLUDED)%2C(text%3AMapwize%2CselectionType%3AINCLUDED)%2C(text%3ARocos%2520%2528Acquired%2520by%2520DroneDeploy%2529%2CselectionType%3AINCLUDED)%2C(text%3AJuntos%2520Global%2CselectionType%3AINCLUDED)%2C(text%3AAPPRL%2CselectionType%3AINCLUDED)%2C(text%3AAlgorithmia%2CselectionType%3AINCLUDED)%2C(text%3ASpeedTree%2520%2528IDV%252C%2520inc%2529%2CselectionType%3AINCLUDED)%2C(text%3ASketchfab%2CselectionType%3AINCLUDED)%2C(text%3APartsSource%2520Inc.%2CselectionType%3AINCLUDED)%2C(text%3AUsermind%2CselectionType%3AINCLUDED)%2C(text%3AApolicy%2520%2528Acquired%2520by%2520Sysdig%2529%2CselectionType%3AINCLUDED)%2C(text%3AVertebrae%252C%2520a%2520Snap%2520Inc.%2520company%2CselectionType%3AINCLUDED)%2C(text%3AAvast%2CselectionType%3AINCLUDED)%2C(text%3ACrew%2CselectionType%3AINCLUDED)%2C(text%3ASymmetry%2520Software%2CselectionType%3AINCLUDED)%2C(text%3AUbiquity6%2520Inc.%2520%2528acquired%2520by%2520Discord%2529%2CselectionType%3AINCLUDED)%2C(text%3AProcureNow%2CselectionType%3AINCLUDED)%2C(text%3APixyz%2520%2528a%2520Unity%2520Technologies%2520brand%2529%2CselectionType%3AINCLUDED)%2C(text%3ASecuredTouch%2520%2528acquired%2520by%2520Ping%2520Identity%2529%2CselectionType%3AINCLUDED)%2C(text%3Ak6%2CselectionType%3AINCLUDED)%2C(text%3AArdius%2CselectionType%3AINCLUDED)%2C(text%3ArideOS%2520%2528acquired%2520by%2520Gopuff%2529%2CselectionType%3AINCLUDED)%2C(text%3ABoomset%2CselectionType%3AINCLUDED)%2C(text%3AModalyst%2CselectionType%3AINCLUDED)%2C(text%3ABigBox%2520VR%2520-%2520A%2520Meta%2520Company%2CselectionType%3AINCLUDED)%2C(text%3ASilexica%2CselectionType%3AINCLUDED)%2C(text%3ADeepMap%252C%2520Inc.%2CselectionType%3AINCLUDED)%2C(text%3AInsent%2520%2528A%2520ZoomInfo%2520Company%2529%2CselectionType%3AINCLUDED)%2C(text%3ASensu%2CselectionType%3AINCLUDED)%2C(text%3AUnifyID%2CselectionType%3AINCLUDED)%2C(text%3AChargeback%2CselectionType%3AINCLUDED)%2C(text%3ASmokescreen%2520%255BAcquired%2520by%2520Zscaler%255D%2CselectionType%3AINCLUDED)%2C(text%3AUnfolded%2CselectionType%3AINCLUDED)%2C(text%3AFeaturePeek%2520%2528acquired%2520by%2520Netlify%2529%2CselectionType%3AINCLUDED)%2C(text%3ATruSTAR%2520%2528acquired%2520by%2520Splunk%2529%2CselectionType%3AINCLUDED)%2C(text%3AKenna%2520Security%2CselectionType%3AINCLUDED)%2C(text%3AIteratively%252C%2520an%2520Amplitude%2520Company%2CselectionType%3AINCLUDED)%2C(text%3AFossID%2CselectionType%3AINCLUDED)%2C(text%3ASedona%2520Systems%2520%2528now%2520part%2520of%2520Cisco%2529%2CselectionType%3AINCLUDED)%2C(text%3APulsar%2520AI%2520Georgia%2CselectionType%3AINCLUDED)%2C(text%3ALightstep%2CselectionType%3AINCLUDED)%2C(text%3AINDUS.AI%2520%2528Acquired%2520by%2520Procore%2520Technologies%2529%2CselectionType%3AINCLUDED)%2C(text%3ASourceDNA%2CselectionType%3AINCLUDED)%2C(text%3ASameplan%2520%2528acquired%2520by%2520Outreach%2529%2CselectionType%3AINCLUDED)%2C(text%3AWoven%2CselectionType%3AINCLUDED)%2C(text%3Askew.%2CselectionType%3AINCLUDED)%2C(text%3AArtStation%2CselectionType%3AINCLUDED)%2C(text%3ATaxJar%2520%257C%2520A%2520Stripe%2520company%2CselectionType%3AINCLUDED)%2C(text%3AGlide%2CselectionType%3AINCLUDED)%2C(text%3APixel8%2CselectionType%3AINCLUDED)%2C(text%3AUpchain%2CselectionType%3AINCLUDED)%2C(text%3AEnjoyHQ%2520%2528now%2520part%2520of%2520UserTesting%2529%2CselectionType%3AINCLUDED)%2C(text%3ASecretHub%2CselectionType%3AINCLUDED)%2C(text%3ARoot%2520AI%2CselectionType%3AINCLUDED)%2C(text%3A24sessions%252C%2520a%2520Bird%2520company%2CselectionType%3AINCLUDED)%2C(text%3APerpule%2CselectionType%3AINCLUDED)%2C(text%3APactSafe%2520%2528acquired%2520by%2520Ironclad%2529%2CselectionType%3AINCLUDED)%2C(text%3ACloud%2520Elements%252C%2520a%2520UiPath%2520company%2CselectionType%3AINCLUDED)%2C(text%3APana%2CselectionType%3AINCLUDED)%2C(text%3AKindite%2520%2528Acquired%2520by%2520RingCentral%2529%2CselectionType%3AINCLUDED)%2C(text%3AZaius%2520%2528now%2520Optimizely%2520-%2520Page%2520Inactive%2529%2CselectionType%3AINCLUDED)%2C(text%3AKaruna%2520Health%2520%2528acquired%2520by%2520Commure%252C%2520Inc.%2529%2CselectionType%3AINCLUDED)%2C(text%3ADropbox%2520DocSend%2CselectionType%3AINCLUDED)%2C(text%3ACurv%2520%2528Acquired%2520by%2520PayPal%2529%2CselectionType%3AINCLUDED)%2C(text%3ASignRequest%2CselectionType%3AINCLUDED)%2C(text%3ABison%2520Trails%2520%2528Acquired%2520by%2520Coinbase%2529%2CselectionType%3AINCLUDED)%2C(text%3AMavens%2520-%2520A%2520Komodo%2520Health%2520Company%2CselectionType%3AINCLUDED)%2C(text%3AIgneous%2520%2528acquired%2520by%2520Rubrik%2529%2CselectionType%3AINCLUDED)%2C(text%3AJuniper%2520Labs%2CselectionType%3AINCLUDED)%2C(text%3APixie%2520Labs%2CselectionType%3AINCLUDED)%2C(text%3ADataform%2CselectionType%3AINCLUDED)%2C(text%3AWorkpath%2CselectionType%3AINCLUDED)%2C(text%3AJuniper%2520Apstra%2CselectionType%3AINCLUDED)%2C(text%3Aimimobile%2520%2528part%2520of%2520Webex%2529%2CselectionType%3AINCLUDED)%2C(text%3ATCP%2527s%2520Humanity%2520Scheduling%2CselectionType%3AINCLUDED)%2C(text%3ASmash.gg%2CselectionType%3AINCLUDED)%2C(text%3AElement%2520AI%252C%2520a%2520ServiceNow%2520company%2CselectionType%3AINCLUDED)%2C(text%3ARespond%2520Software%2CselectionType%3AINCLUDED)%2C(text%3ABanzai%2520Cloud%2CselectionType%3AINCLUDED)%2C(text%3AFyde%2CselectionType%3AINCLUDED)%2C(text%3AVoca.ai%2520%2528Acquired%2520by%2520Snap%2520Inc.%2529%2CselectionType%3AINCLUDED)%2C(text%3AExpanse%252C%2520acquired%2520by%2520Palo%2520Alto%2520Networks%2CselectionType%3AINCLUDED)%2C(text%3AShopKeep%252C%2520a%2520Lightspeed%2520Company%2CselectionType%3AINCLUDED)%2C(text%3AEverString%2CselectionType%3AINCLUDED)%2C(text%3ACimatron%2CselectionType%3AINCLUDED)%2C(text%3ASymphonic%2520Software%2520%2528Now%2520part%2520of%2520Ping%2520Identity%2529%2CselectionType%3AINCLUDED)%2C(text%3ALLamasoft%2520Japan%2CselectionType%3AINCLUDED)%2C(text%3APlumbr%2520%2528acquired%2520by%2520Splunk%2529%2CselectionType%3AINCLUDED)%2C(text%3ARigor%2520%2528acquired%2520by%2520Splunk%2529%2CselectionType%3AINCLUDED)%2C(text%3A128%2520Technology%2CselectionType%3AINCLUDED)%2C(text%3AAxiomSL%2520Connect%2CselectionType%3AINCLUDED)%2C(text%3AArista%2520Security%2CselectionType%3AINCLUDED)%2C(text%3ADeepCode%2520%2528The%2520Software%2520Revolution%2529%2520by%2520Snyk.io%2CselectionType%3AINCLUDED)%2C(text%3ALemnis%2520Technologies%2CselectionType%3AINCLUDED)%2C(text%3ADrone.io%2CselectionType%3AINCLUDED)%2C(text%3ALiveoak%2520Technologies%252C%2520a%2520DocuSign%2520Company%2CselectionType%3AINCLUDED)%2C(text%3AReady%2520At%2520Dawn%2520Studios%2CselectionType%3AINCLUDED)%2C(text%3ASweagle%2CselectionType%3AINCLUDED)%2C(text%3ABigTeam%2CselectionType%3AINCLUDED)%2C(text%3ALastline%252C%2520Inc.%2CselectionType%3AINCLUDED)%2C(text%3APeak%2CselectionType%3AINCLUDED)%2C(text%3AinkFrog%2CselectionType%3AINCLUDED)%2C(text%3AProStar%2520-%2520developers%2520of%2520PointMan%2CselectionType%3AINCLUDED)%2C(text%3AReply.ai%2CselectionType%3AINCLUDED)%2C(text%3AKeybase%2CselectionType%3AINCLUDED)%2C(text%3AMotiv%2520Inc%2CselectionType%3AINCLUDED)%2C(text%3ATeston%2520%2528acquired%2520by%2520UserTesting%2529%2CselectionType%3AINCLUDED)%2C(text%3Anpm%252C%2520Inc.%2CselectionType%3AINCLUDED)%2C(text%3AProcessOut%2CselectionType%3AINCLUDED)%2C(text%3APassage%2520AI%2CselectionType%3AINCLUDED)%2C(text%3AArtomatix%2520%2528Acquired%2520by%2520Unity%2520Technologies%2529%2CselectionType%3AINCLUDED)%2C(text%3ACleverFiles%2CselectionType%3AINCLUDED)%2C(text%3APaxata%2520%2528a%2520DataRobot%2520Company%2529%2CselectionType%3AINCLUDED)%2C(text%3A%2528%25EC%25A3%25BC%2529%2520%25EC%259A%25B0%25EC%2595%2584%25ED%2595%259C%25ED%2598%2595%25EC%25A0%259C%25EB%2593%25A4%2520%2528Woowa%2520Bros.%2529%2CselectionType%3AINCLUDED)%2C(text%3ASpectral%2520Edge%2CselectionType%3AINCLUDED)%2C(text%3ASheltr%2CselectionType%3AINCLUDED)%2C(text%3Aaiden.ai%2CselectionType%3AINCLUDED)%2C(text%3AJASK%2520%2528now%2520Sumo%2520Logic%2529%2CselectionType%3AINCLUDED)%2C(text%3AScout%2520RFP%2CselectionType%3AINCLUDED)%2C(text%3AObserveIT%2CselectionType%3AINCLUDED)%2C(text%3AIOpipe%2CselectionType%3AINCLUDED)%2C(text%3Adataxu%2CselectionType%3AINCLUDED)%2C(text%3AStreamlio%2520%2528acquired%2520by%2520Splunk%2529%2CselectionType%3AINCLUDED)%2C(text%3ASophos%2CselectionType%3AINCLUDED)%2C(text%3AChameleonX%2520%2528Acquired%2520by%2520Akamai%2520Technologies%2529%2CselectionType%3AINCLUDED)%2C(text%3AFeature%2520Labs%252C%2520Inc.%2CselectionType%3AINCLUDED)%2C(text%3AMedallia%2CselectionType%3AINCLUDED)%2C(text%3AHangar%2520Technology%2CselectionType%3AINCLUDED)%2C(text%3AServicefriend%2520%2528acquired%2520by%2520Facebook%2529%2CselectionType%3AINCLUDED)%2C(text%3ASemmle%2CselectionType%3AINCLUDED)%2C(text%3AHedvig%2CselectionType%3AINCLUDED)%2C(text%3AArcadia%2520Data%2CselectionType%3AINCLUDED)%2C(text%3AOmnition%2CselectionType%3AINCLUDED)%2C(text%3ACisco%2520Webex%2520Experience%2520Management%2520%2528formerly%2520CloudCherry%2529%2CselectionType%3AINCLUDED)%2C(text%3APivotal%2520Software%252C%2520Inc.%2CselectionType%3AINCLUDED)%2C(text%3AParallelM%2520-%2520Acquired%2520by%2520DataRobot%2CselectionType%3AINCLUDED)%2C(text%3AMighty%2520AI%2CselectionType%3AINCLUDED)%2C(text%3Adrive.ai%2CselectionType%3AINCLUDED)%2C(text%3AImperva%2520%2528Formerly%2520Distil%2520Networks%2529%2CselectionType%3AINCLUDED)%2C(text%3AImanis%2520Data%2CselectionType%3AINCLUDED)%2C(text%3APendo.io%2CselectionType%3AINCLUDED)%2C(text%3AGitPrime%2CselectionType%3AINCLUDED)%2C(text%3AElectric%2520Cloud%2520%2528acquired%2520by%2520CloudBees%2529%2CselectionType%3AINCLUDED)%2C(text%3AExari%2520Systems%2CselectionType%3AINCLUDED)%2C(text%3ALaserlike%2520Inc%2CselectionType%3AINCLUDED)%2C(text%3AAzuqua%2CselectionType%3AINCLUDED)%2C(text%3AMist%2520Systems%2CselectionType%3AINCLUDED)%2C(text%3ACursor%2CselectionType%3AINCLUDED)%2C(text%3ABlockspring%2CselectionType%3AINCLUDED)%2C(text%3ACamerai%2520%2528acquired%2529%2CselectionType%3AINCLUDED)%2C(text%3ABuildingConnected%2CselectionType%3AINCLUDED)%2C(text%3ASapho%2520%2528acquired%2520by%2520Citrix%2529%2CselectionType%3AINCLUDED)%2C(text%3ATalari%2520%2528Acquired%2520by%2520Oracle%2529%2CselectionType%3AINCLUDED)%2C(text%3AKoko%2CselectionType%3AINCLUDED)%2C(text%3ALiaison%2520Technologies%2CselectionType%3AINCLUDED)%2C(text%3AMeetingbird%2CselectionType%3AINCLUDED)%2C(text%3ASendGrid%2CselectionType%3AINCLUDED)%2C(text%3AmLab%2CselectionType%3AINCLUDED)%2C(text%3ARocket%2520Software%2CselectionType%3AINCLUDED)%2C(text%3AGlint%2CselectionType%3AINCLUDED)%2C(text%3AHortonworks%2CselectionType%3AINCLUDED)%2C(text%3AGradescope%2CselectionType%3AINCLUDED)%2C(text%3ALytmus%2520Inc.%2520%2528acquired%2520by%2520Checkr%2529%2CselectionType%3AINCLUDED)%2C(text%3AAstro%2520Technology%2520%2528acquired%2520by%2520Slack%2529%2CselectionType%3AINCLUDED)%2C(text%3ALobe%2CselectionType%3AINCLUDED)%2C(text%3ATeleporter%2520Realities%2520Inc.%2CselectionType%3AINCLUDED)%2C(text%3ATrustPath%2520%25F0%259F%25A4%2596%2520%25E2%259C%2585%2CselectionType%3AINCLUDED)%2C(text%3ACloudHealth%2520by%2520VMware%2CselectionType%3AINCLUDED)%2C(text%3ASaaSfocus%2CselectionType%3AINCLUDED)%2C(text%3AFieldbook%2CselectionType%3AINCLUDED)%2C(text%3ADistributed%2520Masonry%2520%255Bacquired%2520by%2520Confluent%255D%2CselectionType%3AINCLUDED)%2C(text%3AFrame%2CselectionType%3AINCLUDED)%2C(text%3AMojo%2520Networks%252C%2520Inc.%2CselectionType%3AINCLUDED)%2C(text%3ASolebit%2520%2528Acquired%2520by%2520Mimecast%2529%2CselectionType%3AINCLUDED)%2C(text%3AImperva%2520%2528Formerly%2520Prevoty%2529%2CselectionType%3AINCLUDED)%2C(text%3AScaleFT%252C%2520the%2520Zero%2520Trust%2520security%2520company%2CselectionType%3AINCLUDED)%2C(text%3ADatorama%252C%2520a%2520Salesforce%2520Company%2CselectionType%3AINCLUDED)%2C(text%3AEmburse%2520Abacus%2CselectionType%3AINCLUDED)%2C(text%3ACA%2520Technologies%2CselectionType%3AINCLUDED)%2C(text%3ANexosis%2CselectionType%3AINCLUDED)%2C(text%3ACisco%2520Spaces%2CselectionType%3AINCLUDED)%2C(text%3AEmpirical%2520Systems%2CselectionType%3AINCLUDED)%2C(text%3AVictorOps%2CselectionType%3AINCLUDED)%2C(text%3AGitHub%2CselectionType%3AINCLUDED)%2C(text%3ANoteninja%2520%2528acquired%2520by%2520SalesLoft%2529%2CselectionType%3AINCLUDED)%2C(text%3AZettle%2520by%2520PayPal%2CselectionType%3AINCLUDED)%2C(text%3AOptymyze%2CselectionType%3AINCLUDED)%2C(text%3ARiverbed%2520Technology%2CselectionType%3AINCLUDED)%2C(text%3ABeyondTrust%2CselectionType%3AINCLUDED)%2C(text%3AThe%2520Zulip%2520Open%2520Source%2520Project%2CselectionType%3AINCLUDED)%2C(text%3AForcepoint%2CselectionType%3AINCLUDED)%2C(text%3AHabana%2520Labs%2CselectionType%3AINCLUDED)%2C(text%3ALogMeIn%2CselectionType%3AINCLUDED)%2C(text%3AARENA%2520ONE%2CselectionType%3AINCLUDED)%2C(text%3ABay%2520Dynamics%2CselectionType%3AINCLUDED)%2C(text%3ASegasec%2520%2528Acq.%2520by%2520Mimecast%2529%2CselectionType%3AINCLUDED)%2C(text%3AAppSheet%2CselectionType%3AINCLUDED)%2C(text%3ACloudneeti%2520%2528Acquired%2520by%2520Zscaler%2529%2CselectionType%3AINCLUDED)%2C(text%3AIdaptive%2520%2528acquired%2520by%2520CyberArk%2529%2CselectionType%3AINCLUDED)%2C(text%3AOctarine%2CselectionType%3AINCLUDED)%2C(text%3AEdgewise%2520Networks%2CselectionType%3AINCLUDED)%2C(text%3ACyberX%2CselectionType%3AINCLUDED)%2C(text%3APaladion%2520an%2520Eviden%2520business%2CselectionType%3AINCLUDED)%2C(text%3ADatrium%2CselectionType%3AINCLUDED)%2C(text%3AOP%25C4%2580Q%2CselectionType%3AINCLUDED)%2C(text%3ASpell%2520Security%2520%2528%2520a%2520Qualys%2520Company%2520%2529%2CselectionType%3AINCLUDED)%2C(text%3AUndefined%2520Labs%2CselectionType%3AINCLUDED)%2C(text%3AArm%2CselectionType%3AINCLUDED)%2C(text%3APreempt%252C%2520A%2520CrowdStrike%2520Company%2CselectionType%3AINCLUDED)%2C(text%3AFlowmill%2520%2528acquired%2520by%2520Splunk%2529%2CselectionType%3AINCLUDED)%2C(text%3AInteliSecure%252C%2520a%2520division%2520of%2520Proofpoint%2CselectionType%3AINCLUDED)%2C(text%3ACloudKnox%2520Security%2520Inc.%2520%2528Acquired%2520by%2520Microsoft%2520Corporation%2529%2CselectionType%3AINCLUDED)%2C(text%3ABraintrace%2CselectionType%3AINCLUDED)%2C(text%3ASingular%2520Key%2520%2528now%2520part%2520of%2520Ping%2520Identity%2529%2CselectionType%3AINCLUDED)%2C(text%3ANew%2520Relic%2520CodeStream%2CselectionType%3AINCLUDED)%2C(text%3AArctic%2520Wolf%2520Incident%2520Response%252C%2520Formerly%2520Tetra%2520Defense%2CselectionType%3AINCLUDED)%2C(text%3ALIFARS%252C%2520LLC%2520-%2520a%2520SecurityScorecard%2520Cybersecurity%2520Company%2CselectionType%3AINCLUDED)%2C(text%3ACloudflare%2520Area%25201%2520Security%2CselectionType%3AINCLUDED)%2C(text%3AMandiant%2520%2528part%2520of%2520Google%2520Cloud%2529%2CselectionType%3AINCLUDED)%2C(text%3AWhitehat%2520Security%2CselectionType%3AINCLUDED)%2C(text%3AHdiv%2520Security%2CselectionType%3AINCLUDED)%2C(text%3ASolvvy%2520from%2520Zoom%2CselectionType%3AINCLUDED)%2C(text%3AVMware%2CselectionType%3AINCLUDED)%2C(text%3ASaferPass%2520Password%2520Manager%252C%2520by%2520F-Secure%2CselectionType%3AINCLUDED)%2C(text%3APerimeterX%2520%2528now%2520HUMAN%2529%2CselectionType%3AINCLUDED)%2C(text%3AInfiot%2CselectionType%3AINCLUDED)%2C(text%3ASeekret%2CselectionType%3AINCLUDED)%2C(text%3ACrowdStrike%2520Falcon%2520Surface%2520%2528formerly%2520Reposify%2529%2CselectionType%3AINCLUDED)%2C(text%3AShiftRight%2520%2528Acquired%2520by%2520Zscaler%2529%2CselectionType%3AINCLUDED)%2C(text%3ACloudcraft%2CselectionType%3AINCLUDED)%2C(text%3ATwinWave%2520Security%2520%2528acquired%2520by%2520Splunk%2529%2CselectionType%3AINCLUDED)%2C(text%3ACider%2520Security%2520%2528acquired%2520by%2520Palo%2520Alto%2520Networks%2529%2CselectionType%3AINCLUDED)%2C(text%3APachyderm%2520Inc.%2520%2528Acquired%2520by%2520HPE%2529%2CselectionType%3AINCLUDED)%2C(text%3AMagnet%2520Forensics%2CselectionType%3AINCLUDED)%2C(text%3AValtix%2520%2528now%2520part%2520of%2520Cisco%2529%2CselectionType%3AINCLUDED)%2C(text%3ACanonic%2520Security%2520%2528Acquired%2520by%2520Zscaler%2529%2CselectionType%3AINCLUDED)%2C(text%3AMoogsoft%2CselectionType%3AINCLUDED)%2C(text%3ALaminar%2520%2528acquired%2520by%2520Rubrik%2529%2CselectionType%3AINCLUDED)%2C(text%3ATenable%2CselectionType%3AINCLUDED)%2C(text%3ARedkite%2CselectionType%3AINCLUDED)%2C(text%3ASpera%2520Security%2520%2528by%2520Okta%2529%2CselectionType%3AINCLUDED)%2C(text%3AArmis%2CselectionType%3AINCLUDED)%2C(text%3AGem%2520Security%2520%2528acquired%2520by%2520Wiz%2529%2CselectionType%3AINCLUDED)%2C(text%3ABastionZero%2520%2528acquired%2520by%2520Cloudflare%2529%2CselectionType%3AINCLUDED)%2C(text%3ASecurityScorecard%2CselectionType%3AINCLUDED)%2C(text%3ADigitalOcean%2CselectionType%3AINCLUDED)%2C(text%3AJamf%2CselectionType%3AINCLUDED)%2C(text%3AZscaler%2CselectionType%3AINCLUDED)%2C(text%3ACisco%2CselectionType%3AINCLUDED)%2C(text%3AProofpoint%2CselectionType%3AINCLUDED)%2C(text%3ASalesforce%2CselectionType%3AINCLUDED)%2C(text%3AMimecast%2CselectionType%3AINCLUDED)%2C(text%3AAkamai%2520Technologies%2CselectionType%3AINCLUDED)%2C(text%3AOpenText%2CselectionType%3AINCLUDED)%2C(text%3ABugcrowd%2CselectionType%3AINCLUDED)))%2C(type%3AREGION%2Cvalues%3AList((id%3A102713980%2Ctext%3AIndia%2CselectionType%3AEXCLUDED)))%2C(type%3AFUNCTION%2Cvalues%3AList((id%3A1%2Ctext%3AAccounting%2CselectionType%3AEXCLUDED)%2C(id%3A12%2Ctext%3AHuman%2520Resources%2CselectionType%3AEXCLUDED)%2C(id%3A14%2Ctext%3ALegal%2CselectionType%3AEXCLUDED)%2C(id%3A15%2Ctext%3AMarketing%2CselectionType%3AEXCLUDED)%2C(id%3A2%2Ctext%3AAdministrative%2CselectionType%3AEXCLUDED)%2C(id%3A23%2Ctext%3AReal%2520Estate%2CselectionType%3AEXCLUDED)%2C(id%3A26%2Ctext%3ACustomer%2520Success%2520and%2520Support%2CselectionType%3AEXCLUDED)%2C(id%3A3%2Ctext%3AArts%2520and%2520Design%2CselectionType%3AEXCLUDED)))%2C(type%3ACOMPANY_HEADQUARTERS%2Cvalues%3AList((id%3A103644278%2Ctext%3AUnited%2520States%2CselectionType%3AINCLUDED)%2C(id%3A100506914%2Ctext%3AEurope%2CselectionType%3AINCLUDED)%2C(id%3A101174742%2Ctext%3ACanada%2CselectionType%3AINCLUDED)%2C(id%3A101620260%2Ctext%3AIsrael%2CselectionType%3AINCLUDED)%2C(id%3A102393603%2Ctext%3AAsia%2CselectionType%3AEXCLUDED)%2C(id%3A102713980%2Ctext%3AIndia%2CselectionType%3AEXCLUDED)%2C(id%3A103323778%2Ctext%3AMexico%2CselectionType%3AINCLUDED)%2C(id%3A103537801%2Ctext%3AAfrica%2CselectionType%3AEXCLUDED)%2C(id%3A104305776%2Ctext%3AUnited%2520Arab%2520Emirates%2CselectionType%3AEXCLUDED)%2C(id%3A106057199%2Ctext%3ABrazil%2CselectionType%3AINCLUDED)))%2C(type%3AYEARS_IN_CURRENT_POSITION%2Cvalues%3AList((id%3A1%2Ctext%3ALess%2520than%25201%2520year%2CselectionType%3AINCLUDED)))%2C(type%3APROFILE_LANGUAGE%2Cvalues%3AList((id%3Aen%2Ctext%3AEnglish%2CselectionType%3AINCLUDED)))%2C(type%3ALEAD_INTERACTIONS%2Cvalues%3AList((id%3ALIMP%2Ctext%3AMessaged%2CselectionType%3AEXCLUDED)))%2C(type%3ACURRENT_TITLE%2Cvalues%3AList((id%3A35%2Ctext%3AFounder%2CselectionType%3AINCLUDED)%2C(id%3A103%2Ctext%3ACo-Founder%2CselectionType%3AINCLUDED)%2C(id%3A1128%2Ctext%3ABuilder%2CselectionType%3AINCLUDED)%2C(id%3A153%2Ctext%3AChief%2520Technology%2520Officer%2CselectionType%3AINCLUDED)%2C(id%3A8%2Ctext%3AChief%2520Executive%2520Officer%2CselectionType%3AINCLUDED))))%2Ckeywords%3A%2522something%2520new%2522%2520OR%2520%2522stealth%2522)&sessionId=R7vv8AiYRXiqMseY1m3CwQ%3D%3D
"""

### Part 1: Sales Nav Scraper

In [ ]:
# Sales Nav Scraping
driver = instantiate_driver()
already_scraped_urls, to_scrape_urls = check_and_load_pickles(historyPickle, toScrapePickle)
to_scrape_urls = scrape_profiles(driver, SALES_NAV_SEARCH_URL, already_scraped_urls, to_scrape_urls, toScrapePickle)
print(str(len(to_scrape_urls)) + "profiles identified for scraping") 

### Part 2: Profile scraper, export data to Google Sheets, and reset results pickle

In [ ]:
# Profile Scraping
deduplicate(toScrapePickle, to_scrape_urls, already_scraped_urls)
results = scrape_all_profiles(driver, historyPickle, toScrapePickle, resultsPickle, failedURLsTextFile)


# Export results to Google Sheets
worksheet_title = 'Acquired'
resultsPickle = 'db/acquired_results.pickle'
results = readPickle(resultsPickle)
if results:
    worksheet_url = export_sheet(results, spreadsheet, worksheet_title)
    print(f"Worksheet updated: {worksheet_url}")
else:
    print("No data to export.")


# Reset results
results = []
writePickle(resultsPickle, results)
driver.close()